Importar bibliotecas
###### 3. metricas de avaliação
### 5. Melhorias aplicaveis
* 5.1 tratamento de nulos
* 5.2 balanceamento do target (undersample)
* 5.3 Tratamento de multicolinearidade
* 5.4 tratamento de outliers
* 5.5 seleção de variaveis
* 5.5.1 seleção de variaveis stepwise (Pandas)
* 5.5.2. Analise de p valor
* 5.6 Adição de novas variáveis
###6. Avaliar as melhorias
* 6.1. Melhorias individualmente
### 7. Resultado Final
* 7.1. Recuperar os melhores modelos com hiperparametros
* 7.2. Aplicar melhores modelos nas melhorias aplicáveis com melhor ganho
* 7.3. Avaliar melhores modelos nas melhorias aplicáveis com melhor ganho

##Importar bibliotecas

In [3]:
from pyspark.sql.functions import datediff
from pyspark.sql.functions import to_date
from pyspark.sql.functions import lit
from pyspark.sql.functions import current_date
from pyspark.sql.functions import month
from pyspark.sql.functions import months_between
from pyspark.sql.functions import max
from pyspark.sql.functions import min
from pyspark.sql.functions import year
from pyspark.sql.functions import when
from pyspark.sql.functions import col
from pyspark.sql.functions import desc
from pyspark.sql.functions import mean
from pyspark.sql.functions import countDistinct
from pyspark.sql.functions import count
from pyspark.sql.functions import length
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.tuning import TrainValidationSplit
from pyspark.ml.classification import LogisticRegression, LogisticRegressionModel
from pyspark.ml.classification import DecisionTreeClassifier, DecisionTreeClassificationModel, RandomForestClassifier, GBTClassifier, RandomForestClassificationModel, GBTClassificationModel
from pyspark.ml.evaluation import BinaryClassificationEvaluator
import pandas as pd
from pyspark.ml import Pipeline,Transformer
from pyspark.sql import DataFrame
from typing import Iterable
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.regression import LinearRegressionSummary
from pyspark.ml.classification import LinearSVC,LinearSVCModel
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.stat import Correlation
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# 3. metricas de avaliação

In [5]:
sqlContext.setConf("spark.sql.shuffle.partitions", "2")
sqlContext.setConf("spark.default.parallelism", "2")
print(sqlContext.getConf("spark.default.parallelism"))
print(sqlContext.getConf("spark.sql.shuffle.partitions"))

2
2

In [6]:
df_tabelao = spark.read.parquet('/FileStore/olist/ENTREGA_2/0_bases/ABT_com_nulo.parquet').cache().coalesce(2)

In [7]:
def metricas(modelo,base):
  """
  Função para retornar as metricas de avaliação do modelo.
  """
  predicao = modelo.transform(base)
  #MulticlassClassificationEvaluator param for metric name in evaluation (supports "f1" (default), "weightedPrecision", "weightedRecall", "accuracy")
  accuracy = MulticlassClassificationEvaluator(labelCol='target',metricName='accuracy')
  Precision = MulticlassClassificationEvaluator(labelCol='target',metricName='weightedPrecision')
  Recall = MulticlassClassificationEvaluator(labelCol='target',metricName='weightedRecall')
  #BinaryClassificationEvaluator param for metric name in evaluation (supports "areaUnderROC" (default), "areaUnderPR")
  UnderROC = BinaryClassificationEvaluator(labelCol='target',metricName='areaUnderROC')
  UnderPR = BinaryClassificationEvaluator(labelCol='target',metricName='areaUnderPR')
  
  acuracia = accuracy.evaluate(predicao)
  Precision = Precision.evaluate(predicao)
  Recall = Recall.evaluate(predicao)
  roc = UnderROC.evaluate(predicao)
  PR = UnderPR.evaluate(predicao)
  
  return acuracia,Precision,Recall,roc,PR

In [8]:
def criar_spark_dataframe_por_dicionario(dicionario):
  colunas = list(dicionario.keys())
  nlin = len(dicionario[colunas[0]])
  lista_nova = []
  linhas = []
  for i in range(nlin):
    for j in colunas:
      lista_nova.append(dicionario[j][i])
    print(lista_nova)
    linhas.append(tuple(lista_nova))
    lista_nova = []
  return spark.createDataFrame(linhas,colunas)

In [9]:
def df_metricas(modelos, bases, nome_bases = ['treino', 'teste']):
  bases = zip(bases,nome_bases)
  dados_modelos_bases = {'base': list(),'modelo': list(), 'acuracia': list(), 
                                'Precision': list(), 'Recall': list(), "underROC": list(), "UnderPR": list()}
  for base in bases:
    for modelo in modelos :
      #calcula as métricas
      acuracia,Precision,Recall,roc,PR = metricas(modelo,base[0])
      #ajusta o nome do modelo para compor o dataframe de resposta
      nome_modelo = str(modelo).split()[0].split("_")[0]
      dados_modelos_bases['base'].append(base[1])
      dados_modelos_bases['modelo'].append(nome_modelo)
      dados_modelos_bases['acuracia'].append(round(acuracia,5))
      dados_modelos_bases['Precision'].append(round(Precision,5))
      dados_modelos_bases['Recall'].append(round(Recall,5))
      dados_modelos_bases['underROC'].append(round(roc,5))
      dados_modelos_bases['UnderPR'].append(round(PR,5))
  return criar_spark_dataframe_por_dicionario(dados_modelos_bases)

######modelos que serão usados em todos os testes

In [11]:
svm = LinearSVC(labelCol='target')
gbt = GBTClassifier(labelCol='target')
randforest = RandomForestClassifier(labelCol='target')
reglog = LogisticRegression(maxIter=10, regParam=0.1, labelCol='target')
dectree = DecisionTreeClassifier(labelCol = 'target', maxDepth=5, impurity='gini')

# 5. Melhorias aplicaveis

## 5.1 tratamento de nulos

In [14]:
class Trata_nulos(Transformer):
  def __init__(self, colocar_zeros = False):
    super(Trata_nulos,self).__init__() 
    self.dados_nulos = {'coluna': list(), 
                        'quantidade': list(), 
                        'percent': list(),
                        'tratativa': list()
                       }
    self.colocar_zeros = colocar_zeros
    
  def transform(self, base):
    self.dados_nulos['coluna'] = list()
    self.dados_nulos['quantidade'] = list()
    self.dados_nulos['percent'] = list()
    base.cache().coalesce(2)#.repartition(2)
    print("*****  Análise de nulos  *****")
    tamanho = base.count()
    for col in base.columns:
      qtd_nulos = base.filter(base[col].isNull()).count()
      if qtd_nulos > 0 :
        percent = round(qtd_nulos/tamanho*100,2)
        self.dados_nulos['coluna'].append(col)
        self.dados_nulos['quantidade'].append(qtd_nulos)
        self.dados_nulos['percent'].append(percent)
        if percent < 35 :
          if self.colocar_zeros:
            base = base.fillna(0,subset=[col])
            self.dados_nulos['tratativa'].append('Tratada')
          else:
            mediana = float(base[[col]].summary('50%').collect()[0][1])
            self.dados_nulos['tratativa'].append('Tratada')
            base = base.fillna(mediana,subset=[col])
        else:
          self.dados_nulos['tratativa'].append('Excluida')
          base = base.drop(col)
    
    print(criar_spark_dataframe_por_dicionario(self.dados_nulos))
    print("*****  Fim Análise de nulos  *****")
    return base 

In [15]:
trata_nulos = Trata_nulos()
df_abt_nulos = trata_nulos.transform(df_tabelao)
# Aplicar o sector assembler
assembler = VectorAssembler(
    inputCols = df_abt_nulos.columns[:-1] , 
    outputCol = 'features')
df_abt_nulos = assembler.transform(df_abt_nulos)
treino_nulos, teste_nulos = df_abt_nulos.randomSplit([0.7, 0.3], seed=42)
treino_nulos.cache().coalesce(2)#.repartition(2)
teste_nulos.cache().coalesce(2)#.repartition(2)
treino_nulos.write.mode("overwrite").parquet('/FileStore/olist/ENTREGA_3/1_modelos_com_melhorias_propostas/treino_nulos_mediana.parquet')
teste_nulos.write.mode("overwrite").parquet('/FileStore/olist/ENTREGA_3/1_modelos_com_melhorias_propostas/teste_nulos_mediana.parquet')

***** Análise de nulos *****
['media_dias_entre_delivered_e_review', 39, 0.04, 'Tratada']
['media_dias_receber_e_responder_review', 84, 0.09, 'Tratada']
['max_dias_receber_e_responder_review', 84, 0.09, 'Tratada']
['max_dias_entre_delivered_e_review', 39, 0.04, 'Tratada']
['media_review_comment_message_length', 54709, 58.61, 'Excluida']
['media_review_comment_title_length', 82183, 88.05, 'Excluida']
['max_review_comment_title_length', 82183, 88.05, 'Excluida']
['max_review_comment_message_length', 54709, 58.61, 'Excluida']
DataFrame[coluna: string, quantidade: bigint, percent: double, tratativa: string]
***** Fim Análise de nulos *****

In [16]:
trata_nulos2 = Trata_nulos(colocar_zeros=True)
df_abt_nulos2 = trata_nulos2.transform(df_tabelao)
assembler = VectorAssembler(
    inputCols = df_abt_nulos2.columns[:-1] , 
    outputCol = 'features')
df_abt_nulos2 = assembler.transform(df_abt_nulos2)
treino_nulos2, teste_nulos2 = df_abt_nulos2.randomSplit([0.7, 0.3], seed=42)
df_abt_nulos2.cache().coalesce(2)#.repartition(2)
treino_nulos2.cache().coalesce(2)#.repartition(2)
#~30 seg
treino_nulos2.write.mode("overwrite").parquet('/FileStore/olist/ENTREGA_3/1_modelos_com_melhorias_propostas/treino_nulos_com_zero.parquet')
teste_nulos2.write.mode("overwrite").parquet('/FileStore/olist/ENTREGA_3/1_modelos_com_melhorias_propostas/teste_nulos_com_zero.parquet')

***** Análise de nulos *****
['media_dias_entre_delivered_e_review', 39, 0.04, 'Tratada']
['media_dias_receber_e_responder_review', 84, 0.09, 'Tratada']
['max_dias_receber_e_responder_review', 84, 0.09, 'Tratada']
['max_dias_entre_delivered_e_review', 39, 0.04, 'Tratada']
['media_review_comment_message_length', 54709, 58.61, 'Excluida']
['media_review_comment_title_length', 82183, 88.05, 'Excluida']
['max_review_comment_title_length', 82183, 88.05, 'Excluida']
['max_review_comment_message_length', 54709, 58.61, 'Excluida']
DataFrame[coluna: string, quantidade: bigint, percent: double, tratativa: string]
***** Fim Análise de nulos *****

In [17]:
# Modelo linear  Support Vector Machine (classifier)
modelo_nulos_svm = svm.fit(treino_nulos)
modelo_nulos_svm2 = svm.fit(treino_nulos2)

# Modelo Gradient-boosted tree classifier
modelo_nulos_gbt = gbt.fit(treino_nulos)
modelo_nulos_gbt2 = gbt.fit(treino_nulos2)

# Modelo Random forest classifier
modelo_nulos_randforest = randforest.fit(treino_nulos)
modelo_nulos_randforest2 = randforest.fit(treino_nulos2)

# Modelo Logistic Regression
modelo_nulos_reglog = reglog.fit(treino_nulos)
modelo_nulos_reglog2 = reglog.fit(treino_nulos2)

# Modelo Decision Tree
modelo_nulos_dectree = dectree.fit(treino_nulos)
modelo_nulos_dectree2 = dectree.fit(treino_nulos2)

# #11.39 minutes

* métricas no treino e teste com nulos preenchidos com a **mediana**
* colunas removidas:
    - media_review_comment_message_length
    - media_review_comment_title_length
    - max_review_comment_title_length
    - max_review_comment_message_length

In [19]:
modelos = [modelo_nulos_reglog, 
           modelo_nulos_dectree, 
           modelo_nulos_svm,
           modelo_nulos_randforest,
           modelo_nulos_gbt]
bases = [treino_nulos,teste_nulos]
nome_bases = ['treino_nulos_com_mediana', 'teste_nulos_com_mediana']
df_metricas_treino_teste_nulo_com_mediana =  df_metricas(modelos, bases, nome_bases)
display(df_metricas_treino_teste_nulo_com_mediana)
# #~10 min
# #~1.99 minutes

base,modelo,acuracia,Precision,Recall,underROC,UnderPR
treino_nulos_com_mediana,LogisticRegressionModel:,0.81327,0.81327,0.81327,0.64276,0.45019
treino_nulos_com_mediana,DecisionTreeClassificationModel,0.81884,0.82338,0.81884,0.39666,0.18473
treino_nulos_com_mediana,LinearSVC,0.81488,0.80281,0.81488,0.60251,0.42471
treino_nulos_com_mediana,RandomForestClassificationModel,0.81846,0.82354,0.81846,0.64452,0.45887
treino_nulos_com_mediana,GBTClassificationModel,0.82133,0.83192,0.82133,0.65953,0.48184
teste_nulos_com_mediana,LogisticRegressionModel:,0.8127,0.81089,0.8127,0.63986,0.43951
teste_nulos_com_mediana,DecisionTreeClassificationModel,0.81768,0.82127,0.81768,0.40131,0.1853
teste_nulos_com_mediana,LinearSVC,0.81438,0.8021,0.81438,0.5955,0.41342
teste_nulos_com_mediana,RandomForestClassificationModel,0.81804,0.82318,0.81804,0.64009,0.45197
teste_nulos_com_mediana,GBTClassificationModel,0.81912,0.82691,0.81912,0.64611,0.45862


In [20]:
df_metricas_treino_teste_nulo_com_mediana.write.format('csv').option('header',True).mode('overwrite').option('sep',',').save('/FileStore/olist/ENTREGA_3/1_modelos_com_melhorias_propostas/3_Trata_nulos/Comparacao_metricas_treino_teste_nulo_com_mediana.csv')

* métricas no treino e teste com nulos preenchidos com **zero**
* colunas removidas:
    - media_review_comment_message_length
    - media_review_comment_title_length
    - max_review_comment_title_length
    - max_review_comment_message_length

In [22]:
modelos = [modelo_nulos_reglog2, 
           modelo_nulos_dectree2, 
           modelo_nulos_svm2,
           modelo_nulos_randforest2,
           modelo_nulos_gbt2]
bases = [treino_nulos2,teste_nulos2]
nome_bases = ['treino_nulos_com_zero', 'teste_nulos_com_zero']
df_metricas_treino_teste_nulo_com_zero =  df_metricas(modelos, bases,nome_bases)
display(df_metricas_treino_teste_nulo_com_zero)
# #~10min

base,modelo,acuracia,Precision,Recall,underROC,UnderPR
treino_nulos_com_zero,LogisticRegressionModel:,0.81337,0.81325,0.81337,0.6429,0.45013
treino_nulos_com_zero,DecisionTreeClassificationModel,0.81899,0.82338,0.81899,0.39624,0.18449
treino_nulos_com_zero,LinearSVC,0.81493,0.8028,0.81493,0.60534,0.42292
treino_nulos_com_zero,RandomForestClassificationModel,0.8189,0.82434,0.8189,0.64736,0.45972
treino_nulos_com_zero,GBTClassificationModel,0.82132,0.83145,0.82132,0.65932,0.48135
teste_nulos_com_zero,LogisticRegressionModel:,0.81237,0.81052,0.81237,0.64053,0.44034
teste_nulos_com_zero,DecisionTreeClassificationModel,0.81765,0.82163,0.81765,0.40076,0.18538
teste_nulos_com_zero,LinearSVC,0.8142,0.80207,0.8142,0.5996,0.41158
teste_nulos_com_zero,RandomForestClassificationModel,0.81836,0.82454,0.81836,0.64234,0.45239
teste_nulos_com_zero,GBTClassificationModel,0.8189,0.82734,0.8189,0.64511,0.45835


In [23]:
df_metricas_treino_teste_nulo_com_zero.write.format('csv').option('header',True).mode('overwrite').option('sep',',').save('/FileStore/olist/ENTREGA_3/1_modelos_com_melhorias_propostas/3_Trata_nulos/Comparacao_metricas_treino_teste_nulo_com_zero.csv')

In [24]:
%fs
ls /FileStore/olist/ENTREGA_3/1_modelos_com_melhorias_propostas/3_Trata_nulos/

path,name,size
dbfs:/FileStore/olist/ENTREGA_3/1_modelos_com_melhorias_propostas/3_Trata_nulos/Comparacao_metricas_treino_teste_nulo_com_mediana.csv/,Comparacao_metricas_treino_teste_nulo_com_mediana.csv/,0
dbfs:/FileStore/olist/ENTREGA_3/1_modelos_com_melhorias_propostas/3_Trata_nulos/Comparacao_metricas_treino_teste_nulo_com_zero.csv/,Comparacao_metricas_treino_teste_nulo_com_zero.csv/,0


In [25]:
df_metricas_treino_teste_nulo_com_mediana = spark.read.csv('/FileStore/olist/ENTREGA_3/1_modelos_com_melhorias_propostas/3_Trata_nulos/Comparacao_metricas_treino_teste_nulo_com_mediana.csv', header=True)
df_metricas_treino_teste_nulo_com_zero = spark.read.csv('/FileStore/olist/ENTREGA_3/1_modelos_com_melhorias_propostas/3_Trata_nulos/Comparacao_metricas_treino_teste_nulo_com_zero.csv', header=True)
df_metricas_treino_teste_nulos = df_metricas_treino_teste_nulo_com_mediana.union(df_metricas_treino_teste_nulo_com_zero)

In [26]:
display(df_metricas_treino_teste_nulos)

base,modelo,acuracia,Precision,Recall,underROC,UnderPR
treino_nulos_com_mediana,RandomForestClassificationModel,0.81846,0.82354,0.81846,0.64452,0.45887
treino_nulos_com_mediana,GBTClassificationModel,0.82133,0.83192,0.82133,0.65953,0.48184
teste_nulos_com_mediana,RandomForestClassificationModel,0.81804,0.82318,0.81804,0.64009,0.45197
teste_nulos_com_mediana,GBTClassificationModel,0.81912,0.82691,0.81912,0.64611,0.45862
treino_nulos_com_mediana,DecisionTreeClassificationModel,0.81884,0.82338,0.81884,0.39666,0.18473
teste_nulos_com_mediana,DecisionTreeClassificationModel,0.81768,0.82127,0.81768,0.40131,0.1853
treino_nulos_com_mediana,LogisticRegressionModel:,0.81327,0.81327,0.81327,0.64276,0.45019
teste_nulos_com_mediana,LogisticRegressionModel:,0.8127,0.81089,0.8127,0.63986,0.43951
treino_nulos_com_mediana,LinearSVC,0.81488,0.80281,0.81488,0.60251,0.42471
teste_nulos_com_mediana,LinearSVC,0.81438,0.8021,0.81438,0.5955,0.41342


In [27]:
display(df_metricas_treino_teste_nulos)

base,modelo,acuracia,Precision,Recall,underROC,UnderPR
treino_nulos_com_mediana,RandomForestClassificationModel,0.81846,0.82354,0.81846,0.64452,0.45887
treino_nulos_com_mediana,GBTClassificationModel,0.82133,0.83192,0.82133,0.65953,0.48184
teste_nulos_com_mediana,RandomForestClassificationModel,0.81804,0.82318,0.81804,0.64009,0.45197
teste_nulos_com_mediana,GBTClassificationModel,0.81912,0.82691,0.81912,0.64611,0.45862
treino_nulos_com_mediana,DecisionTreeClassificationModel,0.81884,0.82338,0.81884,0.39666,0.18473
teste_nulos_com_mediana,DecisionTreeClassificationModel,0.81768,0.82127,0.81768,0.40131,0.1853
treino_nulos_com_mediana,LogisticRegressionModel:,0.81327,0.81327,0.81327,0.64276,0.45019
teste_nulos_com_mediana,LogisticRegressionModel:,0.8127,0.81089,0.8127,0.63986,0.43951
treino_nulos_com_mediana,LinearSVC,0.81488,0.80281,0.81488,0.60251,0.42471
teste_nulos_com_mediana,LinearSVC,0.81438,0.8021,0.81438,0.5955,0.41342


In [28]:
#Concluímos que não tem diferença no nosso problema se tratarmos nulos com zero ou com mediana

In [29]:
#como não tem diferença significativa no tratamento de nulos, vamos tratar com fillna(0) para deixar mais independentes as modificações abaixo
df_tabelao_fillna_zero = df_tabelao.fillna(0).cache().coalesce(2)

## 5.2 balanceamento do target (undersample)

In [31]:
class Undersampling_class(Transformer):
    """
    A custom Transformer which drops all columns that have at least one of the
    words from the banned_list in the name.
    """
    def __init__(self):
        super(Undersampling_class, self).__init__()
        
        
    def transform(self, base, nome_target = 'target', seed=42):
        #divide a base em dois dataframes cada um com um target diferente
        base_target_0 = base.filter(f"{nome_target} = 0")
        base_target_1 = base.filter(f"{nome_target} = 1")
        #verifica a proporcao de cada target para poder balancear
        tgt = base.select(f"{nome_target}").groupby(f"{nome_target}").count()
        tgt_vetor = tgt.rdd.flatMap(lambda x: x).collect()
        #print(tgt_vetor)
        tgt_vetor_dict = {str(tgt_vetor[0]):tgt_vetor[1], str(tgt_vetor[2]):tgt_vetor[3]}
        #[1, 20385, 0, 72957]
        print(tgt_vetor_dict)
        prop = 1
        #target 0 é maior que target 1
        if tgt_vetor_dict["0"]>tgt_vetor_dict["1"]:
          print("zeros maior que 1")
          #calcula qual a porcentagen de 1 que corresponde a mesma qtd de 0
          prop = tgt_vetor_dict["1"]/tgt_vetor_dict["0"]
          print(f"prop = {prop}")
          #pega uma amostra do target 1 de forma que fique na mesma proporcao do target 0
          base_sample = base_target_0.sample(withReplacement=False, fraction=prop, seed=seed)
          #une a target 0 com a amostra de target 1 de forma a aproximar 50% de cada 
          return base_target_1.union(base_sample)
        #target 0 é maior que target 1
        else:
          #calcula qual a porcentagen de 0 que corresponde a mesma qtd de 1
          prop = tgt_vetor_dict["1"]/tgt_vetor_dict["0"]
          print(f"prop = {prop}")
          #pega uma amostra do target 0 de forma que fique na mesma proporcao do target 1
          base_sample = base_target_1.sample(withReplacement=False, fraction=prop, seed=seed)
          #une a target 1 com a amostra de target 0 de forma a aproximar 50% de cada 
          return base_target_0.union(base_sample)

In [32]:
# Balancear a base
Balancear = Undersampling_class()
df_balanceado = Balancear.transform(df_tabelao_fillna_zero)
#display(df_balanceado)

{'0': 72957, '1': 20385}
zeros maior que 1
prop = 0.2794111599983552

In [33]:
# Plotar resultado
fig, ax =plt.subplots(1,2,figsize=((14, 4)),sharex=True)
sns.set(style="white")
sns.barplot(x='target',y='count',data=df_balanceado.select(f"target").groupby(f"target").count().toPandas(),
            ax=ax[0],palette="rocket") 
ax[0].axhline(0,color="k", clip_on=False)
ax[0].set_ylabel("")
ax[0].set_title('Base Balanceada')
sns.barplot(x='target',y='count',data=df_tabelao_fillna_zero.select(f"target").groupby(f"target").count().toPandas(),
            ax=ax[1],palette="rocket") 
ax[1].axhline(0,color="k", clip_on=False)
ax[1].set_ylabel("")
ax[1].set_title('Base Original')

sns.despine(bottom=True)

In [34]:
# Aplicar o sector assembler
assembler = VectorAssembler(
    inputCols = df_balanceado.columns[:-1] , 
    outputCol = 'features')

In [35]:
df_abt_balanceado = assembler.transform(df_balanceado)
treino_balanceado, teste_balanceado = df_abt_balanceado.randomSplit([0.7, 0.3], seed=42)
treino_balanceado.cache().coalesce(2)#.repartition(2)
teste_balanceado.cache().coalesce(2)#.repartition(2)
treino_balanceado.write.mode("overwrite").parquet('/FileStore/olist/ENTREGA_3/1_modelos_com_melhorias_propostas/treino_balanceado.parquet')
teste_balanceado.write.mode("overwrite").parquet('/FileStore/olist/ENTREGA_3/1_modelos_com_melhorias_propostas/teste_balanceado.parquet')

In [36]:
# Modelo linear  Support Vector Machine (classifier)
modelo_balanceado_svm = svm.fit(treino_balanceado)

# Modelo Gradient-boosted tree classifier
modelo_balanceado_gbt = gbt.fit(treino_balanceado)

# Modelo Random forest classifier
modelo_balanceado_randforest = randforest.fit(treino_balanceado)

# Modelo Logistic Regression
modelo_balanceado_reglog = reglog.fit(treino_balanceado)

# Modelo Decision Tree
modelo_balanceado_dectree = dectree.fit(treino_balanceado)

#51.08 minutes sem cache().repartition(2)
#9.05 minutes com cache().repartition(2)

In [37]:
#define os modelos que vão ser avaliads
modelos = [modelo_balanceado_reglog, modelo_balanceado_dectree, modelo_balanceado_svm,
          modelo_balanceado_randforest,modelo_balanceado_gbt]
#define as bases que serão avaliadas
bases = [treino_balanceado, teste_balanceado]
#se não for base treino e teste devemos passar mais um parâmetro com o nome das bases
nome_bases = ['treino_balanceado','teste_balanceado']
df_metricas_treino_teste_balanceado =  df_metricas(modelos, bases, nome_bases)
display(df_metricas_treino_teste_balanceado)

base,modelo,acuracia,Precision,Recall,underROC,UnderPR
treino_balanceado,LogisticRegressionModel:,0.7153,0.72719,0.7153,0.76571,0.79562
treino_balanceado,DecisionTreeClassificationModel,0.71272,0.73207,0.71272,0.29691,0.38315
treino_balanceado,LinearSVC,0.71213,0.73311,0.71213,0.75635,0.78807
treino_balanceado,RandomForestClassificationModel,0.71509,0.73282,0.71509,0.76959,0.79979
treino_balanceado,GBTClassificationModel,0.72334,0.73984,0.72334,0.78365,0.81846
teste_balanceado,LogisticRegressionModel:,0.71338,0.72666,0.71338,0.76386,0.79694
teste_balanceado,DecisionTreeClassificationModel,0.70953,0.72999,0.70953,0.30125,0.3875
teste_balanceado,LinearSVC,0.70854,0.73062,0.70854,0.75036,0.78736
teste_balanceado,RandomForestClassificationModel,0.711,0.72987,0.711,0.7662,0.79985
teste_balanceado,GBTClassificationModel,0.71724,0.73437,0.71724,0.77368,0.80889


In [38]:
df_metricas_treino_teste_balanceado.write.format('csv').option('header',True).mode('overwrite').option('sep',',').save('/FileStore/olist/ENTREGA_3/1_modelos_com_melhorias_propostas/0_Balancear/Comparacao_metricas_treino_teste_balanceado.csv')

In [39]:
%fs
ls /FileStore/olist/ENTREGA_3/1_modelos_com_melhorias_propostas/0_Balancear

path,name,size
dbfs:/FileStore/olist/ENTREGA_3/1_modelos_com_melhorias_propostas/0_Balancear/Comparacao_metricas_treino_teste_balanceado.csv/,Comparacao_metricas_treino_teste_balanceado.csv/,0
dbfs:/FileStore/olist/ENTREGA_3/1_modelos_com_melhorias_propostas/0_Balancear/Comparacao_metricas_treino_teste_multicolinearidade.csv/,Comparacao_metricas_treino_teste_multicolinearidade.csv/,0


In [40]:
df_metricas_treino_teste_balanceado = spark.read.csv('/FileStore/olist/ENTREGA_3/1_modelos_com_melhorias_propostas/0_Balancear/Comparacao_metricas_treino_teste_balanceado.csv', header=True)

In [41]:
display(df_metricas_treino_teste_balanceado)

base,modelo,acuracia,Precision,Recall,underROC,UnderPR
treino_balanceado,RandomForestClassificationModel,0.71509,0.73282,0.71509,0.76959,0.79979
treino_balanceado,GBTClassificationModel,0.72334,0.73984,0.72334,0.78365,0.81846
teste_balanceado,RandomForestClassificationModel,0.711,0.72987,0.711,0.7662,0.79985
teste_balanceado,GBTClassificationModel,0.71724,0.73437,0.71724,0.77368,0.80889
treino_balanceado,DecisionTreeClassificationModel,0.71272,0.73207,0.71272,0.29691,0.38315
teste_balanceado,DecisionTreeClassificationModel,0.70953,0.72999,0.70953,0.30125,0.3875
teste_balanceado,LogisticRegressionModel:,0.71338,0.72666,0.71338,0.76386,0.79694
treino_balanceado,LogisticRegressionModel:,0.7153,0.72719,0.7153,0.76571,0.79562
treino_balanceado,LinearSVC,0.71213,0.73311,0.71213,0.75635,0.78807
teste_balanceado,LinearSVC,0.70854,0.73062,0.70854,0.75036,0.78736


In [42]:
display(df_metricas_treino_teste_balanceado)

base,modelo,acuracia,Precision,Recall,underROC,UnderPR
treino_balanceado,RandomForestClassificationModel,0.93572,0.927,0.93572,0.75411,0.36797
treino_balanceado,GBTClassificationModel,0.93926,0.93052,0.93926,0.76934,0.42449
teste_balanceado,RandomForestClassificationModel,0.93371,0.92321,0.93371,0.75393,0.35388
teste_balanceado,GBTClassificationModel,0.93549,0.92334,0.93549,0.76979,0.36693
treino_balanceado,DecisionTreeClassificationModel,0.93658,0.9253,0.93658,0.29367,0.04773
teste_balanceado,DecisionTreeClassificationModel,0.93417,0.92113,0.93417,0.28692,0.04811
treino_balanceado,LogisticRegressionModel:,0.93224,0.92326,0.93224,0.7598,0.35426
teste_balanceado,LogisticRegressionModel:,0.93034,0.9186,0.93034,0.76086,0.35164
treino_balanceado,LinearSVC,0.93375,0.91986,0.93375,0.68046,0.28345
teste_balanceado,LinearSVC,0.93183,0.91627,0.93183,0.68192,0.27358


Se olharmos somente esta etapa 5.2, os melhores modelos seriam GBTClassifier e LogisticRegressionModel.
Porem nenhum deles foi melhor que a baseline

## 5.3 Tratamento de multicolinearidade

In [45]:
class seleciona_por_correlacao(Transformer):
  """
  A custom Transformer which drops all columns selected by correlation process.
  """
  
  def __init__(self):
    #usa o construtor da classe pai (Transformer)
    super(seleciona_por_correlacao, self).__init__()
    #atributos, não tem parêntese pra chamar
    self.colunas_retiradas = set()
    self.colunas_mantidas = set()
    self.df_correlation = None
  
  #método, tem parêntese pra chamar
  def matrix_correlacao(self, df_usar):
    self.df_correlation = None
    df_usar.cache().coalesce(2)#.repartition(2)
    df_usar = df_usar.fillna(0)
    vector_col = "variaveis"
    assembler = VectorAssembler(inputCols=df_usar.columns, 
                                outputCol=vector_col)
    df_vector = assembler.transform(df_usar)
    matrix = Correlation.corr(df_vector, vector_col)
    DenseMatrix = matrix.head()[0]
    DenseMatrix_to_numpy_ndarray = DenseMatrix.toArray()
    ndarray_to_list = DenseMatrix_to_numpy_ndarray.tolist()
    self.df_correlation = spark.createDataFrame(ndarray_to_list,df_usar.columns)
    
  
  def trata_multicolinearida(self, df_usar):
    #o método matrix_correlacao faz o cálculo da correlação entre todos os elemetos da base e "cria" o atributo self.df_correlation
    self.matrix_correlacao(df_usar)
    df_correlation_collected = self.df_correlation.collect()
    #excluir = list()
    tamanho = len(df_correlation_collected)-1
    lista_seleção = range(tamanho)
    for col_x in lista_seleção:
      for col_y in lista_seleção:
        if col_x == col_y:
          break
        correl = df_correlation_collected[col_x][col_y]
        if correl > 0.9:
          correl_x = df_correlation_collected[col_x][tamanho]
          correl_y = df_correlation_collected[col_y][tamanho]
          if correl_x > correl_y :
            #excluir.append(col_y)
            self.colunas_retiradas.add(col_y)
            self.colunas_mantidas.add(col_x)
          elif correl_y > correl_x :
            self.colunas_retiradas.add(col_x)
            self.colunas_mantidas.add(col_y)
            #excluir.append(col_x)
          else:
            self.colunas_retiradas.add(col_y)
            self.colunas_mantidas.add(col_x)
            #excluir.append(col_y)

  def transform(self, df_usar):
    print("*****  Análise de correlação  *****")
    
# #     tratativa para coluna de vetor
#     feature = False
#     for col , tipo in treino.dtypes:
#       if tipo == 'vector':
#         df_usar = df_usar.drop(col)
#         feature = col

    self.trata_multicolinearida(df_usar)
    colunas = df_usar.columns
    print(f'Colunas deletadas:' )
    for i in self.colunas_retiradas:
      print(f"\t{colunas[i]}")
      df_usar = df_usar.drop(colunas[i])
      
#     if feature:
#       assembler = VectorAssembler(
#           inputCols = df_usar.columns[:-1] , 
#           outputCol = 'features')
#       df_usar = assembler.transform(df_usar)

    print("*****  Fim Análise de correlação  *****")
    #retorna o dataframe com as colunas já retiradas
    return df_usar

In [46]:
# Tratar a base 
multicolinearidade = seleciona_por_correlacao()
df_multicolinearidade = multicolinearidade.transform(df_tabelao_fillna_zero)

***** Análise de correlação *****
Colunas deletadas:
	soma_orders_approved_year_2016
	soma_orders_4_trimestre_2016
	soma_orders_3_trimestre_2018
	media_dias_entre_delivered_e_review
	media_review_comment_message_length
	media_review_comment_title_length
***** Fim Análise de correlação *****

In [47]:
#cria o vetor para poder usar no modelo
assembler = VectorAssembler( inputCols = df_multicolinearidade.columns[:-1] , outputCol = 'features')
df_abt_multicolinearidade = assembler.transform(df_multicolinearidade)

In [48]:
#divide a base em treino e teste com o mesmo seed da entrega 2
treino_multicolinearidade, teste_multicolinearidade = df_abt_multicolinearidade.randomSplit([0.7, 0.3], seed=42)
treino_multicolinearidade = treino_multicolinearidade.cache().coalesce(2)#.repartition(2)
teste_multicolinearidade = teste_multicolinearidade.cache().coalesce(2)#.repartition(2)
treino_multicolinearidade.write.mode("overwrite").parquet('/FileStore/olist/ENTREGA_3/1_modelos_com_melhorias_propostas/treino_multicolinearidade.parquet')
teste_multicolinearidade.write.mode("overwrite").parquet('/FileStore/olist/ENTREGA_3/1_modelos_com_melhorias_propostas/teste_multicolinearidade.parquet')

In [49]:
# retornar a matriz de correlação com as variaveis com maior correlação para plotar
col_multicolinearidade = list(multicolinearidade.colunas_mantidas)
col_multicolinearidade.extend(list(multicolinearidade.colunas_retiradas))
#vg_corr = pd.DataFrame(multicolinearidade.df_correlation.collect()).iloc[col_multicolinearidade,col_multicolinearidade]
vg_corr_aux = multicolinearidade.df_correlation.toPandas()
vg_corr = pd.DataFrame(vg_corr_aux).iloc[col_multicolinearidade,col_multicolinearidade]

# gerar o plot de correlação

for i in col_multicolinearidade:
  vg_corr.rename(columns={i: df_tabelao.columns[i]},index={i: df_tabelao.columns[i]},inplace=True)
  
mask = np.zeros_like(vg_corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

plt.figure(figsize=(17, 12))

sns.heatmap(vg_corr, 
            xticklabels = vg_corr.columns.values,
            yticklabels = vg_corr.columns.values,
            cmap='RdBu_r',
            annot = True,
            mask = mask);


In [50]:
# Modelo linear  Support Vector Machine (classifier)
modelo_multicolinearidade_svm = svm.fit(treino_multicolinearidade)

# Modelo Gradient-boosted tree classifier
modelo_multicolinearidade_gbt = gbt.fit(treino_multicolinearidade)

# Modelo Random forest classifier
modelo_multicolinearidade_randforest = randforest.fit(treino_multicolinearidade)

# Modelo Logistic Regression
modelo_multicolinearidade_reglog = reglog.fit(treino_multicolinearidade)

# Modelo Decision Tree
modelo_multicolinearidade_dectree = dectree.fit(treino_multicolinearidade)
#9.48 minutes

In [51]:
#define os modelos que vão ser avaliads
modelos = [modelo_multicolinearidade_reglog, modelo_multicolinearidade_dectree, modelo_multicolinearidade_svm,
          modelo_multicolinearidade_randforest,modelo_multicolinearidade_gbt]
#define as bases que serão avaliadas
bases = [treino_multicolinearidade, teste_multicolinearidade]
#se não for base treino e teste devemos passar mais um parâmetro com o nome das bases
nome_bases = ['treino_multicolinearidade', 'teste_multicolinearidade']
df_metricas_treino_teste_multicolinearidade =  df_metricas(modelos, bases, nome_bases)

['treino_multicolinearidade', 'LogisticRegressionModel:', 0.82362, 0.82114, 0.82362, 0.76339, 0.58423]
['treino_multicolinearidade', 'DecisionTreeClassificationModel', 0.83285, 0.82093, 0.83285, 0.33471, 0.15853]
['treino_multicolinearidade', 'LinearSVC', 0.81565, 0.80384, 0.81565, 0.72464, 0.52589]
['treino_multicolinearidade', 'RandomForestClassificationModel', 0.82119, 0.83066, 0.82119, 0.75603, 0.58524]
['treino_multicolinearidade', 'GBTClassificationModel', 0.83679, 0.82548, 0.83679, 0.77896, 0.61749]
['teste_multicolinearidade', 'LogisticRegressionModel:', 0.82148, 0.81735, 0.82148, 0.75783, 0.57562]
['teste_multicolinearidade', 'DecisionTreeClassificationModel', 0.83027, 0.81747, 0.83027, 0.34077, 0.16007]
['teste_multicolinearidade', 'LinearSVC', 0.81259, 0.79958, 0.81259, 0.71933, 0.51603]
['teste_multicolinearidade', 'RandomForestClassificationModel', 0.81826, 0.82657, 0.81826, 0.74798, 0.57266]
['teste_multicolinearidade', 'GBTClassificationModel', 0.83332, 0.821, 0.83332, 0.76841, 0.59677]

In [52]:
display(df_metricas_treino_teste_multicolinearidade)

base,modelo,acuracia,Precision,Recall,underROC,UnderPR
treino_multicolinearidade,LogisticRegressionModel:,0.82362,0.82114,0.82362,0.76339,0.58423
treino_multicolinearidade,DecisionTreeClassificationModel,0.83285,0.82093,0.83285,0.33471,0.15853
treino_multicolinearidade,LinearSVC,0.81565,0.80384,0.81565,0.72464,0.52589
treino_multicolinearidade,RandomForestClassificationModel,0.82119,0.83066,0.82119,0.75603,0.58524
treino_multicolinearidade,GBTClassificationModel,0.83679,0.82548,0.83679,0.77896,0.61749
teste_multicolinearidade,LogisticRegressionModel:,0.82148,0.81735,0.82148,0.75783,0.57562
teste_multicolinearidade,DecisionTreeClassificationModel,0.83027,0.81747,0.83027,0.34077,0.16007
teste_multicolinearidade,LinearSVC,0.81259,0.79958,0.81259,0.71933,0.51603
teste_multicolinearidade,RandomForestClassificationModel,0.81826,0.82657,0.81826,0.74798,0.57266
teste_multicolinearidade,GBTClassificationModel,0.83332,0.821,0.83332,0.76841,0.59677


In [53]:
df_metricas_treino_teste_multicolinearidade.write.format('csv').option('header',True).mode('overwrite').option('sep',',').save('/FileStore/olist/ENTREGA_3/1_modelos_com_melhorias_propostas/1_multicolinearidade/Comparacao_metricas_treino_teste_multicolinearidade.csv')

In [54]:
%fs
ls /FileStore/olist/ENTREGA_3/1_modelos_com_melhorias_propostas/1_multicolinearidade

path,name,size
dbfs:/FileStore/olist/ENTREGA_3/1_modelos_com_melhorias_propostas/1_multicolinearidade/Comparacao_metricas_treino_teste_multicolinearidade.csv/,Comparacao_metricas_treino_teste_multicolinearidade.csv/,0


In [55]:
df_metricas_treino_teste_multicolinearidade = spark.read.csv('/FileStore/olist/ENTREGA_3/1_modelos_com_melhorias_propostas/1_multicolinearidade/Comparacao_metricas_treino_teste_multicolinearidade.csv', header=True)

In [56]:
display(df_metricas_treino_teste_multicolinearidade)

base,modelo,acuracia,Precision,Recall,underROC,UnderPR
treino_multicolinearidade,RandomForestClassificationModel,0.82119,0.83066,0.82119,0.75603,0.58524
treino_multicolinearidade,GBTClassificationModel,0.83679,0.82548,0.83679,0.77896,0.61749
teste_multicolinearidade,RandomForestClassificationModel,0.81826,0.82657,0.81826,0.74798,0.57266
teste_multicolinearidade,GBTClassificationModel,0.83332,0.821,0.83332,0.76841,0.59677
treino_multicolinearidade,DecisionTreeClassificationModel,0.83285,0.82093,0.83285,0.33471,0.15853
teste_multicolinearidade,DecisionTreeClassificationModel,0.83027,0.81747,0.83027,0.34077,0.16007
treino_multicolinearidade,LogisticRegressionModel:,0.82362,0.82114,0.82362,0.76339,0.58423
teste_multicolinearidade,LogisticRegressionModel:,0.82148,0.81735,0.82148,0.75783,0.57562
treino_multicolinearidade,LinearSVC,0.81565,0.80384,0.81565,0.72464,0.52589
teste_multicolinearidade,LinearSVC,0.81259,0.79958,0.81259,0.71933,0.51603


Se olharmos somente esta etapa 5.3, os melhores modelos seriam GBTClassifier e DecisionTreeClassifier.

GBTClassifier foi melhor que a baseline e o DecisionTreeClassifier foi igual.

## 5.4 tratamento de outliers

In [59]:
class retira_outliers_de_todas_as_variaveis_binarias(Transformer):
    """
    1. verificar a cardinalidade se é 2;
    2. verificar o summary 10% e 90%;
    3. remover as linhas diferentes caso mais de 90% seja só uma categoria;
    4. remove a coluna que agora vai possuir apenas uma categoria.
    """
    def __init__(self):
        super(retira_outliers_de_todas_as_variaveis_binarias, self).__init__()
        self.colunas_retiradas = []
        
    def verificar_cardinalidade(self, df, coluna):
        card = len(df.select(coluna).distinct().collect())
        #print(f"A cardinalidae da coluna {coluna} é {card} .")
        return card

    def verificar_info_10_e_90_percent(self, df, coluna):
        summ = df[[coluna]].summary('10%','90%')
        summ_collect = summ.collect()
        retorno = {summ_collect[0][0]:summ_collect[0][1],summ_collect[1][0]:summ_collect[1][1]}
        #print(f"Os valores de 10% e 90% são : {retorno}")
        return retorno

    def retira_outliers_de_variaveis_binarias(self, df, coluna):
        card = self.verificar_cardinalidade(df,coluna)
        if card == 1:
            self.colunas_retiradas.append(coluna)
            print(f"\tA coluna {coluna} foi removida!")
            return df.drop(coluna)
        if card == 2:
            info = self.verificar_info_10_e_90_percent(df,coluna)
            if info["10%"] == info["90%"]:
                print(f"\tA coluna {coluna} possui pelo menos 90% base com a categoria {info['10%']} !")
                self.colunas_retiradas.append(coluna)
                return df.filter(f"{coluna} = {info['10%']}").drop(coluna)
        else:
            return df 
              
    def transform(self, base):
        base.cache().coalesce(2)#.repartition(2)
        print("*****  Análise de outliers  *****")
        for coluna in base.columns:
            if coluna:
                try:
                  base_nova = self.retira_outliers_de_variaveis_binarias(base,coluna)
                  if base_nova:
                    base = base_nova
                except:
                  pass
        print("*****  Fim Análise de outliers  *****")
        return base

In [60]:
trata_outlier = retira_outliers_de_todas_as_variaveis_binarias()
df_outlier = trata_outlier.transform(df_tabelao_fillna_zero)
# Aplicar o sector assembler
assembler = VectorAssembler(
    inputCols = df_outlier.columns[:-1] , 
    outputCol = 'features')

df_abt_outlier = assembler.transform(df_outlier)
treino_outlier, teste_outlier = df_abt_outlier.randomSplit([0.7, 0.3], seed=42)

treino_outlier = treino_outlier.cache().coalesce(2)#.repartition(2)
teste_outlier = teste_outlier.cache().coalesce(2)#.repartition(2)

treino_outlier.write.mode("overwrite").parquet('/FileStore/olist/ENTREGA_3/1_modelos_com_melhorias_propostas/treino_outlier.parquet')
teste_outlier.write.mode("overwrite").parquet('/FileStore/olist/ENTREGA_3/1_modelos_com_melhorias_propostas/teste_outlier.parquet')

***** Análise de outliers *****
	A coluna soma_orders_2_trimestre_2016 foi removida!
	A coluna soma_orders_3_trimestre_2016 possui pelo menos 90% base com a categoria 0 !
	A coluna soma_orders_1_trimestre_2016 foi removida!
	A coluna soma_orders_4_trimestre_2018 foi removida!
	A coluna soma_orders_1_semestre_2016 foi removida!
***** Fim Análise de outliers *****

In [61]:
# Modelo linear  Support Vector Machine (classifier)
modelo_outlier_svm = svm.fit(treino_outlier)

# Modelo Gradient-boosted tree classifier
modelo_outlier_gbt = gbt.fit(treino_outlier)

# Modelo Random forest classifier
modelo_outlier_randforest = randforest.fit(treino_outlier)

# Modelo Logistic Regression
modelo_outlier_reglog = reglog.fit(treino_outlier)

# Modelo Decision Tree
modelo_outlier_dectree = dectree.fit(treino_outlier)

#13.16 minutes

In [62]:
#define os modelos que vão ser avaliads
modelos = [modelo_outlier_reglog, modelo_outlier_dectree, modelo_outlier_svm,
          modelo_outlier_randforest,modelo_outlier_gbt]
#define as bases que serão avaliadas
bases = [treino_outlier, teste_outlier]
#se não for base treino e teste devemos passar mais um parâmetro com o nome das bases
nome_bases = ['treino_outlier', 'teste_outlier']
df_metricas_treino_teste_outlier =  df_metricas(modelos, bases, nome_bases)
display(df_metricas_treino_teste_outlier)

base,modelo,acuracia,Precision,Recall,underROC,UnderPR
treino_outlier,LogisticRegressionModel:,0.82649,0.8189,0.82649,0.76084,0.57915
treino_outlier,DecisionTreeClassificationModel,0.83416,0.82227,0.83416,0.31373,0.15466
treino_outlier,LinearSVC,0.81508,0.80286,0.81508,0.72368,0.5244
treino_outlier,RandomForestClassificationModel,0.82223,0.83118,0.82223,0.75522,0.5841
treino_outlier,GBTClassificationModel,0.83762,0.82669,0.83762,0.78012,0.61905
teste_outlier,LogisticRegressionModel:,0.82528,0.81644,0.82528,0.76099,0.57827
teste_outlier,DecisionTreeClassificationModel,0.82987,0.8163,0.82987,0.31254,0.15465
teste_outlier,LinearSVC,0.81391,0.80192,0.81391,0.72102,0.51661
teste_outlier,RandomForestClassificationModel,0.81979,0.82796,0.81979,0.75538,0.57744
teste_outlier,GBTClassificationModel,0.83267,0.81998,0.83267,0.77245,0.60061


In [63]:
df_metricas_treino_teste_outlier.write.format('csv').option('header',True).mode('overwrite').option('sep',',').save('/FileStore/olist/ENTREGA_3/1_modelos_com_melhorias_propostas/2_outliers/Comparacao_metricas_treino_teste_outlier.csv')

In [64]:
df_metricas_treino_teste_outlier = spark.read.csv('/FileStore/olist/ENTREGA_3/1_modelos_com_melhorias_propostas/2_outliers/Comparacao_metricas_treino_teste_outlier.csv', header=True)

In [65]:
display(df_metricas_treino_teste_outlier)

base,modelo,acuracia,Precision,Recall,underROC,UnderPR
treino_outlier,RandomForestClassificationModel,0.82223,0.83118,0.82223,0.75522,0.5841
treino_outlier,GBTClassificationModel,0.83762,0.82669,0.83762,0.78012,0.61905
teste_outlier,RandomForestClassificationModel,0.81979,0.82796,0.81979,0.75538,0.57744
teste_outlier,GBTClassificationModel,0.83267,0.81998,0.83267,0.77245,0.60061
treino_outlier,DecisionTreeClassificationModel,0.83416,0.82227,0.83416,0.31373,0.15466
teste_outlier,DecisionTreeClassificationModel,0.82987,0.8163,0.82987,0.31254,0.15465
treino_outlier,LogisticRegressionModel:,0.82649,0.8189,0.82649,0.76084,0.57915
teste_outlier,LogisticRegressionModel:,0.82528,0.81644,0.82528,0.76099,0.57827
treino_outlier,LinearSVC,0.81508,0.80286,0.81508,0.72368,0.5244
teste_outlier,LinearSVC,0.81391,0.80192,0.81391,0.72102,0.51661


Se olharmos somente esta etapa 5.4, os melhores modelos seriam GBTClassifier e DecisionTreeClassifier.

GBTClassifier e DecisionTreeClassifier foram melhores que a baseline.

## 5.5 seleção de variaveis

#### 5.5.1 seleção de variaveis stepwise (Pandas)
Esta etapa deve ser executada somente se a base suportar o toPandas, caso contrario não funciona. 
Na etapa seguinte elaboramos um algoritmo que suporta ser rodado de forma distribuida.

In [69]:
import statsmodels.api as sm

class selecao_stepwise(Transformer):
    
    def __init__(self,lista_inicial=[]):
        super(selecao_stepwise, self).__init__()
        self.incluido = lista_inicial
        
    def selecao_stepwise(self,X, y, 
                       lista_inicial=[], 
                       limite_forward=0.1, 
                       limite_backward = 0.15, 
                       detalhe=True):
    
      """
      Executar uma seleção de variaveis para frente e para trás
      (forward-backward) baseado no valor-p de statsmodels.api.OLS
      Argumentos :

      X - pandas.DataFrame com as variaveis candidatas
      y - variavel resposta 
      lista_inicial - lista de recursos para começar (nomes de colunas de X)

      limite_forward - inclui a variavel se seu valor-p for menor que 'limite_forward'
      limite_backward - exclui a variavel se seu valor-p for maior que 'limite_backward'
      detalhado - se deseja imprimir a sequência de inclusões e exclusões
      retorna - A lista com as variaveis para o modelo

      Sempre defina limite_forward menor que limite_backward para evitar loop infinito.
      mais detalhes em https://en.wikipedia.org/wiki/Stepwise_regression
      """


      #incluido = list(lista_inicial)
      while True:
          mudar=False
          # forward step
          #pega uma lista com todas as variaveis que estão em X e não estão em incluido
          excluido = list(set(X.columns)-set(self.incluido))
          novo_pvalor = pd.Series(index=excluido)
          for nova_coluna in excluido:
              model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[self.incluido+[nova_coluna]]))).fit()
              novo_pvalor[nova_coluna] = model.pvalues[nova_coluna]
          molhor_pvalor = novo_pvalor.min()
          if molhor_pvalor < limite_forward:
              melhor_variavel = novo_pvalor.idxmin()
              #melhor_variavel = novo_pvalor.argmin()
              self.incluido.append(melhor_variavel)
              mudar=True
              if detalhe:
                  print('Add  {:30} with p-value {:.6}'.format(melhor_variavel, molhor_pvalor))

          # backward step
          model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[self.incluido]))).fit()
          # use all coefs except intercept
          pvalor = model.pvalues.iloc[1:]
          pior_pvalor = pvalor.max() # null if pvalor is empty
          if pior_pvalor > limite_backward:
              mudar=True
              pior_variavel = pvalor.idxmax()
              #pior_variavel = pvalor.argmax()
              self.incluido.remove(pior_variavel)
              if detalhe:
                  print('Drop {:30} with p-value {:.6}'.format(pior_variavel, pior_pvalor))
          if not mudar:
              break
    
    def transform(self,base,target = "target",limite_forward=0.1,limite_backward = 0.15,detalhe=True):
      base.cache().coalesce(2)#.repartition(2)
      print("*****  selecao stepwise  *****")
      print("colunas originais:")
      print(base.columns)
      df_abt_pd = base.toPandas()
      print("\nInicia o processo:\n")
      self.selecao_stepwise(df_abt_pd.drop('target',axis=1),df_abt_pd['target'])
      print("\nTerminou o processo:\n")
      if "target" not in self.incluido:
        self.incluido.append("target")
      resposta = base.select(self.incluido)
      print("colunas resultantes:")
      print(resposta.columns)
      print("*****  Fim selecao stepwise  *****")
      return resposta


In [70]:
selecao_stepwise = selecao_stepwise()
df_stepwise = selecao_stepwise.transform(df_tabelao_fillna_zero)
# Aplicar o sector assembler
assembler = VectorAssembler(
    inputCols = df_stepwise.columns[:-1] , 
    outputCol = 'features')

df_abt_stepwise = assembler.transform(df_stepwise)
treino_stepwise, teste_stepwise = df_abt_stepwise.randomSplit([0.7, 0.3], seed=42)
treino_stepwise = treino_stepwise.cache().coalesce(2)#.repartition(2)
teste_stepwise = teste_stepwise.cache().coalesce(2)#.repartition(2)
treino_stepwise.write.mode("overwrite").parquet('/FileStore/olist/ENTREGA_3/1_modelos_com_melhorias_propostas/treino_stepwise.parquet')
teste_stepwise.write.mode("overwrite").parquet('/FileStore/olist/ENTREGA_3/1_modelos_com_melhorias_propostas/teste_stepwise.parquet')

***** selecao stepwise *****
colunas originais:
['soma_orders_regiao_centro_oeste', 'soma_orders_regiao_sudeste', 'soma_orders_regiao_nordeste', 'soma_orders_regiao_sul', 'soma_orders_regiao_norte', 'soma_orders_approved_year_2016', 'soma_orders_approved_year_2017', 'soma_orders_approved_year_2018', 'soma_orders_4_trimestre_2017', 'soma_orders_1_trimestre_2017', 'soma_orders_2_trimestre_2016', 'soma_orders_4_trimestre_2016', 'soma_orders_3_trimestre_2018', 'soma_orders_3_trimestre_2016', 'soma_orders_2_trimestre_2017', 'soma_orders_1_trimestre_2018', 'soma_orders_1_trimestre_2016', 'soma_orders_4_trimestre_2018', 'soma_orders_2_trimestre_2018', 'soma_orders_3_trimestre_2017', 'soma_orders_1_semestre_2017', 'soma_orders_2_semestre_2018', 'soma_orders_1_semestre_2016', 'soma_orders_2_semestre_2017', 'soma_orders_2_semestre_2016', 'soma_orders_1_semestre_2018', 'soma_orders_entrega_atrasada_ate_1_semana', 'soma_orders_entrega_super_atrasada', 'soma_orders_entrega_atrasada_de_1_a_2_semanas', 'soma_orders_entrega_adiantada_ate_8_dias', 'soma_orders_entrega_super_adiantada', 'soma_orders_MA', 'soma_orders_GO', 'soma_orders_CE', 'soma_orders_AC', 'soma_orders_PR', 'soma_orders_DF', 'soma_orders_ES', 'soma_orders_RJ', 'soma_orders_RO', 'soma_orders_SP', 'soma_orders_PB', 'soma_orders_SE', 'soma_orders_RR', 'soma_orders_RS', 'soma_orders_BA', 'soma_orders_PI', 'soma_orders_RN', 'soma_orders_MT', 'soma_orders_AM', 'soma_orders_PA', 'soma_orders_MG', 'soma_orders_AP', 'soma_orders_TO', 'soma_orders_AL', 'soma_orders_PE', 'soma_orders_SC', 'soma_orders_MS', 'media_dias_entre_delivered_e_review', 'media_dias_receber_e_responder_review', 'max_dias_receber_e_responder_review', 'max_dias_entre_delivered_e_review', 'media_review_comment_message_length', 'media_review_comment_title_length', 'max_review_comment_title_length', 'max_review_comment_message_length', 'target']

Inicia o processo:

/databricks/python/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
 return ptp(axis=axis, out=out, **kwargs)
/databricks/python/lib/python3.7/site-packages/statsmodels/base/model.py:1100: RuntimeWarning: invalid value encountered in true_divide
 return self.params / self.bse
/databricks/python/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in greater
 return (self.a < x) & (x < self.b)
/databricks/python/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:877: RuntimeWarning: invalid value encountered in less
 return (self.a < x) & (x < self.b)
/databricks/python/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:1831: RuntimeWarning: invalid value encountered in less_equal
 cond2 = cond0 & (x <= self.a)
Add soma_orders_entrega_super_atrasada with p-value 0.0
Add soma_orders_entrega_atrasada_ate_1_semana with p-value 0.0
Add media_review_comment_message_length with p-value 0.0
Add soma_orders_entrega_atrasada_de_1_a_2_semanas with p-value 0.0
Add soma_orders_entrega_adiantada_ate_8_dias with p-value 2.77821e-34
Add soma_orders_3_trimestre_2018 with p-value 1.60005e-25
Add soma_orders_2_semestre_2018 with p-value 1.60005e-25
Add soma_orders_1_trimestre_2018 with p-value 3.70336e-17
Add soma_orders_SP with p-value 2.73131e-10
Add soma_orders_approved_year_2017 with p-value 1.5688e-10
Add max_review_comment_title_length with p-value 3.80875e-10
Add soma_orders_3_trimestre_2017 with p-value 1.1602e-09
Add max_review_comment_message_length with p-value 1.5124e-06
Drop media_review_comment_message_length with p-value 0.207987
Add soma_orders_RJ with p-value 0.00331464
Add soma_orders_BA with p-value 0.026428
Add soma_orders_GO with p-value 0.0239265
Add soma_orders_PA with p-value 0.0210028
Add soma_orders_MG with p-value 0.0585557
Add soma_orders_RO with p-value 0.0691297
Add media_review_comment_message_length with p-value 0.0887372

Terminou o processo:



In [71]:
# Modelo linear  Support Vector Machine (classifier)
modelo_stepwise_svm = svm.fit(treino_stepwise)

# Modelo Gradient-boosted tree classifier
modelo_stepwise_gbt = gbt.fit(treino_stepwise)

# Modelo Random forest classifier
modelo_stepwise_randforest = randforest.fit(treino_stepwise)

# Modelo Logistic Regression
modelo_stepwise_reglog = reglog.fit(treino_stepwise)

# Modelo Decision Tree
modelo_stepwise_dectree = dectree.fit(treino_stepwise)

In [72]:
#define os modelos que vão ser avaliads
modelos = [modelo_stepwise_reglog, modelo_stepwise_dectree, modelo_stepwise_svm,
          modelo_stepwise_randforest,modelo_stepwise_gbt]
#define as bases que serão avaliadas
bases = [treino_stepwise, teste_stepwise]
#se não for base treino e teste devemos passar mais um parâmetro com o nome das bases
nome_bases = ['treino_stepwise', 'teste_stepwise']
df_metricas_treino_teste_stepwise =  df_metricas(modelos, bases, nome_bases)
display(df_metricas_treino_teste_stepwise)

base,modelo,acuracia,Precision,Recall,underROC,UnderPR
treino_stepwise,LogisticRegressionModel:,0.82698,0.81898,0.82698,0.76208,0.5814
treino_stepwise,DecisionTreeClassificationModel,0.83016,0.81737,0.83016,0.29388,0.15562
treino_stepwise,LinearSVC,0.816,0.80511,0.816,0.73419,0.54218
treino_stepwise,RandomForestClassificationModel,0.8262,0.81277,0.8262,0.75741,0.56608
treino_stepwise,GBTClassificationModel,0.83364,0.82109,0.83364,0.77605,0.606
teste_stepwise,LogisticRegressionModel:,0.82353,0.81397,0.82353,0.76152,0.56996
teste_stepwise,DecisionTreeClassificationModel,0.82798,0.81458,0.82798,0.29881,0.15708
teste_stepwise,LinearSVC,0.81191,0.79663,0.81191,0.72556,0.52329
teste_stepwise,RandomForestClassificationModel,0.82299,0.80815,0.82299,0.75785,0.55536
teste_stepwise,GBTClassificationModel,0.82952,0.81547,0.82952,0.77186,0.5875


In [73]:
df_metricas_treino_teste_stepwise.write.format('csv').option('header',True).mode('overwrite').option('sep',',').save('/FileStore/olist/ENTREGA_3/1_modelos_com_melhorias_propostas/4_stepwise/Comparacao_metricas_treino_teste_stepwise.csv')

In [74]:
df_metricas_treino_teste_stepwise = spark.read.csv('/FileStore/olist/ENTREGA_3/1_modelos_com_melhorias_propostas/4_stepwise/Comparacao_metricas_treino_teste_stepwise.csv', header=True)

In [75]:
display(df_metricas_treino_teste_stepwise)

base,modelo,acuracia,Precision,Recall,underROC,UnderPR
teste_stepwise,RandomForestClassificationModel,0.82299,0.80815,0.82299,0.75785,0.55536
teste_stepwise,GBTClassificationModel,0.82952,0.81547,0.82952,0.77186,0.5875
treino_stepwise,RandomForestClassificationModel,0.8262,0.81277,0.8262,0.75741,0.56608
treino_stepwise,GBTClassificationModel,0.83364,0.82109,0.83364,0.77605,0.606
treino_stepwise,DecisionTreeClassificationModel,0.83016,0.81737,0.83016,0.29388,0.15562
teste_stepwise,DecisionTreeClassificationModel,0.82798,0.81458,0.82798,0.29881,0.15708
treino_stepwise,LogisticRegressionModel:,0.82698,0.81898,0.82698,0.76208,0.5814
teste_stepwise,LogisticRegressionModel:,0.82353,0.81397,0.82353,0.76152,0.56996
teste_stepwise,LinearSVC,0.81191,0.79663,0.81191,0.72556,0.52329
treino_stepwise,LinearSVC,0.816,0.80511,0.816,0.73419,0.54218


Se olharmos somente esta etapa 5.5.1, os melhores modelos seriam GBTClassifier e LogisticRegression.

GBTClassifier não foi melhor que a baseline e LogisticRegression foi melhor que a baseline.

####5.5.2. Analise de p valor

In [78]:
class vetorizar(Transformer):
    """
    Faz o Vector assembler baseado numa base recebida, com isso conseguimos usar no pipeline
    """
    def __init__(self):
        super(vetorizar, self).__init__()
        self.DataFrame_resultante = None
        
    def transform(self, base,verbose=False):
        inputCols_ = list(base.columns)
        inputCols_.remove('target')
        if verbose:
          print("*****  Vetorizando a base  *****")
          print(inputCols_)
        assembler = VectorAssembler( inputCols = inputCols_ , outputCol = 'features')
        self.DataFrame_resultante = assembler.transform(base)
        if verbose:
          print("*****  base vetorizada, pronta para rodar o modelo  *****")
        return self.DataFrame_resultante

In [79]:
def analise_pvalor(df_base,vet,pvalor_maximo=0.15):
  """
  função para remover do modelo variaveis com alto Pvalor (baixa associação ao modelo).
  recebe uma base sem a coluna vetorizada, e um objeto que vetoriza a base.
  """
  from pyspark.ml.regression import GeneralizedLinearRegression
  incluir = list(df_base.columns)
  while True :

    glr = GeneralizedLinearRegression(family="gaussian", regParam=0.3,labelCol='target')
    df_base = vet.transform(df_base)
    # Fit the model
    modelo = glr.fit(df_base)

    #     cria o df para analise
    df_summary = pd.DataFrame({'coluna':df_base.columns[:-2], "coeficiente_glm":  list(modelo.coefficients), 
                               "pvalor": [round(i,5) for i in modelo.summary.pValues[1:]]})

    pvalor_maior = df_summary['pvalor'].max()

    if pvalor_maior < pvalor_maximo or pd.isnull(pvalor_maior):
      break

    remover_coluna = df_summary['coluna'].where(df_summary['pvalor']== pvalor_maior).dropna()
    remover_coluna = remover_coluna.to_string(index=False).split('\n')[0].replace(' ','')

    print(remover_coluna,pvalor_maior)
    df_base = df_base.drop(remover_coluna)
    df_base = df_base.drop("features")
    incluir.remove(remover_coluna)
    
  return df_base


In [80]:
vet = vetorizar()
df_pvalor = analise_pvalor(df_tabelao_fillna_zero,vet)
display(df_pvalor)

soma_orders_MS,media_dias_entre_delivered_e_review,media_dias_receber_e_responder_review,max_dias_receber_e_responder_review,max_dias_entre_delivered_e_review,media_review_comment_message_length,media_review_comment_title_length,max_review_comment_title_length,max_review_comment_message_length,target,features
0,2.0,1.0,-1,2,57.0,0.0,0,57,0,"List(1, 9, List(), List(0.0, 2.0, 1.0, -1.0, 2.0, 57.0, 0.0, 0.0, 57.0))"
0,0.0,0.0,0,0,0.0,0.0,0,0,0,"List(0, 9, List(), List())"
0,2.0,1.0,-1,2,0.0,0.0,0,0,0,"List(0, 9, List(1, 2, 3, 4), List(2.0, 1.0, -1.0, 2.0))"
0,2.0,1.0,-1,2,0.0,0.0,0,0,0,"List(0, 9, List(1, 2, 3, 4), List(2.0, 1.0, -1.0, 2.0))"
0,1.0,0.0,0,1,0.0,0.0,0,0,0,"List(0, 9, List(1, 4), List(1.0, 1.0))"
0,2.0,1.0,-1,2,116.0,8.0,8,116,0,"List(1, 9, List(), List(0.0, 2.0, 1.0, -1.0, 2.0, 116.0, 8.0, 8.0, 116.0))"
0,2.0,0.0,0,2,61.0,0.0,0,61,0,"List(0, 9, List(1, 4, 5, 8), List(2.0, 2.0, 61.0, 61.0))"
0,1.0,0.0,0,1,53.0,17.0,17,53,0,"List(1, 9, List(), List(0.0, 1.0, 0.0, 0.0, 1.0, 53.0, 17.0, 17.0, 53.0))"
0,5.0,4.0,-4,5,0.0,0.0,0,0,0,"List(0, 9, List(1, 2, 3, 4), List(5.0, 4.0, -4.0, 5.0))"
0,2.0,1.0,-1,2,0.0,0.0,0,0,1,"List(0, 9, List(1, 2, 3, 4), List(2.0, 1.0, -1.0, 2.0))"


In [81]:
treino_pvalor, teste_pvalor = df_pvalor.randomSplit([0.7, 0.3], seed=42)
treino_pvalor = treino_pvalor.cache().coalesce(2)#.repartition(2)
teste_pvalor = teste_pvalor.cache().coalesce(2)#.repartition(2)
treino_pvalor.write.mode("overwrite").parquet('/FileStore/olist/ENTREGA_3/1_modelos_com_melhorias_propostas/treino_pvalor.parquet')
teste_pvalor.write.mode("overwrite").parquet('/FileStore/olist/ENTREGA_3/1_modelos_com_melhorias_propostas/teste_pvalor.parquet')

In [82]:
# Modelo linear  Support Vector Machine (classifier)
modelo_pvalor_svm = svm.fit(treino_pvalor)

# Modelo Gradient-boosted tree classifier
modelo_pvalor_gbt = gbt.fit(treino_pvalor)

# Modelo Random forest classifier
modelo_pvalor_randforest = randforest.fit(treino_pvalor)

# Modelo Logistic Regression
modelo_pvalor_reglog = reglog.fit(treino_pvalor)

# Modelo Decision Tree
modelo_pvalor_dectree = dectree.fit(treino_pvalor)


In [83]:
modelos_pvalor = modelos_pvalor = [modelo_pvalor_svm,modelo_pvalor_gbt,modelo_pvalor_randforest,modelo_pvalor_reglog,modelo_pvalor_dectree]
bases_valor = [teste_pvalor,treino_pvalor]
nome_modelo = ['teste_pvalor','treino_pvalor']

df_metricas_treino_teste_pvalor = df_metricas(modelos_pvalor,bases_valor,nome_bases=nome_modelo)
display(df_metricas_treino_teste_pvalor)

base,modelo,acuracia,Precision,Recall,underROC,UnderPR
teste_pvalor,LinearSVC,0.79756,0.81379,0.79756,0.69677,0.51051
teste_pvalor,GBTClassificationModel,0.82995,0.81791,0.82995,0.76313,0.59558
teste_pvalor,RandomForestClassificationModel,0.82912,0.817,0.82912,0.75318,0.58618
teste_pvalor,LogisticRegressionModel:,0.80086,0.78308,0.80086,0.72299,0.49604
teste_pvalor,DecisionTreeClassificationModel,0.82848,0.81583,0.82848,0.31891,0.15919
treino_pvalor,LinearSVC,0.80214,0.81836,0.80214,0.69139,0.50097
treino_pvalor,GBTClassificationModel,0.83545,0.82391,0.83545,0.76445,0.60176
treino_pvalor,RandomForestClassificationModel,0.8317,0.81867,0.8317,0.75119,0.58189
treino_pvalor,LogisticRegressionModel:,0.80462,0.78483,0.80462,0.71744,0.47953
treino_pvalor,DecisionTreeClassificationModel,0.83206,0.8189,0.83206,0.31983,0.15499


In [84]:
df_metricas_treino_teste_pvalor.write.format('csv').option('header',True).mode('overwrite').option('sep',',').save('/FileStore/olist/ENTREGA_3/1_modelos_com_melhorias_propostas/4_stepwise/Comparacao_metricas_treino_teste_pvalor.csv')

In [85]:
df_metricas_treino_teste_pvalor = spark.read.csv('/FileStore/olist/ENTREGA_3/1_modelos_com_melhorias_propostas/4_stepwise/Comparacao_metricas_treino_teste_pvalor.csv', header=True)

In [86]:
display(df_metricas_treino_teste_pvalor)

base,modelo,acuracia,Precision,Recall,underROC,UnderPR
treino_pvalor,LogisticRegressionModel:,0.80462,0.78483,0.80462,0.71744,0.47953
treino_pvalor,DecisionTreeClassificationModel,0.83206,0.8189,0.83206,0.31983,0.15499
teste_pvalor,LogisticRegressionModel:,0.80086,0.78308,0.80086,0.72299,0.49604
teste_pvalor,DecisionTreeClassificationModel,0.82848,0.81583,0.82848,0.31891,0.15919
treino_pvalor,RandomForestClassificationModel,0.8317,0.81867,0.8317,0.75119,0.58189
teste_pvalor,RandomForestClassificationModel,0.82912,0.817,0.82912,0.75318,0.58618
treino_pvalor,GBTClassificationModel,0.83545,0.82391,0.83545,0.76445,0.60176
teste_pvalor,GBTClassificationModel,0.82995,0.81791,0.82995,0.76313,0.59558
treino_pvalor,LinearSVC,0.80214,0.81836,0.80214,0.69139,0.50097
teste_pvalor,LinearSVC,0.79756,0.81379,0.79756,0.69677,0.51051


Se olharmos somente esta etapa 5.5.2, os melhores modelos seriam GBTClassifier, DecisionTreeClassifier e RandomForestClassifier.

GBTClassifier e DecisionTreeClassifier não foram melhores que a baseline e RandomForestClassifier foi levemente melhor que a baseline.

## 5.6 Adição de novas variáveis

In [89]:
df_payments = (spark.read
                .csv('/mnt/datasets/brazilian-ecommerce/olist_order_payments_dataset.csv', 
                     header=True, 
                     sep=',', 
                     inferSchema=True)
                .dropna())
df_orders = (spark.read.csv('/mnt/datasets/brazilian-ecommerce/olist_orders_dataset.csv', header=True, sep=',', inferSchema=True)
             .coalesce(2)#.repartition(2)
             .cache()
             .dropna()
             .dropDuplicates())
df_customers = (spark.read
                .csv('/mnt/datasets/brazilian-ecommerce/olist_customers_dataset.csv', 
                     header=True, 
                     sep=',', 
                     inferSchema=True)
                .dropna())

In [90]:
df_orders_customers = (df_orders[['customer_id','order_id']].join(df_customers[['customer_id','customer_unique_id']], 
                                      df_orders['customer_id'] == df_customers['customer_id'], 
                                      how='left')
                       .drop(df_customers['customer_id'])
                       .drop(df_orders['customer_id']))
df_customers_payments = (df_orders_customers[['customer_unique_id','order_id']].join(df_payments, 
                                      df_orders_customers['order_id'] == df_payments['order_id'], 
                                      how='left')
                       .drop(df_payments['order_id']))

In [91]:
df_customers_payments_datamart = (df_customers_payments
                         .withColumn("pagamento_boleto",
                                     when((col("payment_type") == "boleto"),1).otherwise(0))
                         .withColumn("pagamento_credito", 
                                     when((col("payment_type") == "credit_card"),1).otherwise(0))
                         .withColumn("pagamento_debito",
                                     when((col("payment_type") == "debit_card"),1).otherwise(0))
                         .withColumn("pagamento_voucher",
                                     when((col("payment_type") == "voucher"),1).otherwise(0))
                         .withColumn("pagamento_parcelados",
                                     when((col("payment_installments") >1 ),1).otherwise(0))
                         .drop('payment_installments','payment_type','payment_sequential','order_id')
                        )

In [92]:
df_customers_payments_tr  = (df_customers_payments_datamart.groupBy('customer_unique_id')
                          .agg({'payment_value': 'avg', 'pagamento_boleto': 'sum','pagamento_credito': 'sum',
                                'pagamento_debito': 'sum','pagamento_voucher': 'sum','pagamento_parcelados': 'sum'})
                          .withColumnRenamed('avg(payment_value)', 'media_valor_pagamento')
                          .withColumnRenamed('sum(pagamento_boleto)', 'qtd_pagamento_boleto')
                          .withColumnRenamed('sum(pagamento_credito)', 'qtd_pagamento_credito')
                          .withColumnRenamed('sum(pagamento_debito)', 'qtd_pagamento_debito')
                          .withColumnRenamed('sum(pagamento_voucher)', 'qtd_pagamento_voucher')
                          .withColumnRenamed('sum(pagamento_parcelados)', 'qtd_pagamento_parcelados'))
display(df_customers_payments_tr)

customer_unique_id,qtd_pagamento_boleto,media_valor_pagamento,qtd_pagamento_credito,qtd_pagamento_voucher,qtd_pagamento_parcelados,qtd_pagamento_debito
3a653a41f6f9fc3d2a113cf8398680e8,0,179.12,1,0,1,0
7c142cf63193a1473d2e66489a9ae977,0,72.2,1,0,0,0
932afa1e708222e5821dac9cd5db4cae,0,75.16,1,0,1,0
39382392765b6dc74812866ee5ee92a7,1,35.95,0,0,0,0
299905e3934e9e181bfb2e164dd4b4f8,0,84.88,1,1,0,0
f2a85dec752b8517b5e58a06ff3cd937,0,259.06,1,0,0,0
ccafc1c3f270410521c3c6f3b249870f,0,103.755,2,0,2,0
451e48381edab7f1f6dbfa6d728616ff,0,187.57,1,0,1,0
a6fefcd9f434474cf6fcd8ed1102fd63,0,224.9,1,0,1,0
37363700139c1aef873bbcd916e57dfd,0,63.1,1,0,0,0


In [93]:
df_tabelao2 = spark.read.parquet('/FileStore/olist/ENTREGA_2/0_bases/ABT_com_nulo_e_unique_id.parquet')
df_tabelao_pagamento = (df_tabelao2.join(df_customers_payments_tr,
                                        df_tabelao2['customer_unique_id'] == df_customers_payments_tr['customer_unique_id'], 
                                        how='left')
                        .drop(df_customers_payments_tr['customer_unique_id'])
                        .drop(df_tabelao2['customer_unique_id']))


In [94]:
df_tabelao_pagamento = df_tabelao_pagamento.fillna(0).drop("features")
df_tabelao_pagamento = vetorizar().transform(df_tabelao_pagamento, verbose = False)
treino_add_variaveis, teste_add_variaveis = df_tabelao_pagamento.randomSplit([0.7, 0.3], seed=42)

In [95]:
treino_add_variaveis = treino_add_variaveis.cache().coalesce(2)#.repartition(2)
teste_add_variaveis = teste_add_variaveis.cache().coalesce(2)#.repartition(2)
treino_add_variaveis.write.mode("overwrite").parquet('/FileStore/olist/ENTREGA_3/1_modelos_com_melhorias_propostas/treino_add_variaveis.parquet')
teste_add_variaveis.write.mode("overwrite").parquet('/FileStore/olist/ENTREGA_3/1_modelos_com_melhorias_propostas/teste_add_variaveis.parquet')

In [96]:
# Modelo linear  Support Vector Machine (classifier)
modelo_add_variaveis_svm = svm.fit(treino_add_variaveis)

# Modelo Gradient-boosted tree classifier
modelo_add_variaveis_gbt = gbt.fit(treino_add_variaveis)

# Modelo Random forest classifier
modelo_add_variaveis_randforest = randforest.fit(treino_add_variaveis)

# Modelo Logistic Regression
modelo_add_variaveis_reglog = reglog.fit(treino_add_variaveis)

# Modelo Decision Tree
modelo_add_variaveis_dectree = dectree.fit(treino_add_variaveis)

In [97]:
modelos_add_variaveis = [modelo_add_variaveis_svm,
                         modelo_add_variaveis_gbt,
                         modelo_add_variaveis_randforest,
                         modelo_add_variaveis_reglog,
                         modelo_add_variaveis_dectree]
bases_valor = [treino_add_variaveis,teste_add_variaveis]
nome_modelo = ['treino_add_variaveis','teste_add_variaveis']

df_metricas_treino_teste_add_variaveis = df_metricas(modelos_add_variaveis,bases_valor,nome_bases=nome_modelo)
display(df_metricas_treino_teste_add_variaveis)

base,modelo,acuracia,Precision,Recall,underROC,UnderPR
treino_add_variaveis,LinearSVC,0.81587,0.80421,0.81587,0.71359,0.51047
treino_add_variaveis,GBTClassificationModel,0.83908,0.82854,0.83908,0.7796,0.61968
treino_add_variaveis,RandomForestClassificationModel,0.82436,0.82856,0.82436,0.76204,0.59051
treino_add_variaveis,LogisticRegressionModel:,0.82754,0.81958,0.82754,0.76197,0.5794
treino_add_variaveis,DecisionTreeClassificationModel,0.8344,0.8217,0.8344,0.31187,0.15354
teste_add_variaveis,LinearSVC,0.81204,0.79877,0.81204,0.71612,0.51306
teste_add_variaveis,GBTClassificationModel,0.83215,0.81979,0.83215,0.7724,0.60444
teste_add_variaveis,RandomForestClassificationModel,0.8201,0.82276,0.8201,0.7586,0.58871
teste_add_variaveis,LogisticRegressionModel:,0.82333,0.81459,0.82333,0.76096,0.5766
teste_add_variaveis,DecisionTreeClassificationModel,0.82932,0.81574,0.82932,0.31556,0.15684


In [98]:
df_metricas_treino_teste_add_variaveis.write.format('csv').option('header',True).mode('overwrite').option('sep',',').save('/FileStore/olist/ENTREGA_3/1_modelos_com_melhorias_propostas/5_novas_variaveis/Comparacao_metricas_treino_teste_add_variaveis.csv')

In [99]:
df_metricas_treino_teste_add_variaveis = spark.read.csv('/FileStore/olist/ENTREGA_3/1_modelos_com_melhorias_propostas/5_novas_variaveis/Comparacao_metricas_treino_teste_add_variaveis.csv', header=True)

Se olharmos somente esta etapa 5.6, os melhores modelos seriam GBTClassifier e DecisionTreeClassifier.

GBTClassifier e DecisionTreeClassifier foram melhores que a baseline.

#6. Avaliar as melhorias

#### 6.1. Melhorias individualmente

In [103]:
#O balanceamento não se mostrou efetivo para maximizar  os modelos, por isso foi retirado das análises 

df_metricas_treino_teste_inicial = spark.read.csv('/FileStore/olist/ENTREGA_3/0_modelos_sem_hiperparametros/Comparacao_metricas_treino_teste_inicial.csv', header=True)
df_metricas_treino_teste_hiperparametros = spark.read.csv('/FileStore/olist/ENTREGA_3/2_modelos_com_hiperparametros/Comparacao_metricas_treino_teste_hiperparametros.csv', header=True)
df_metricas_gerais = (df_metricas_treino_teste_inicial
                      .union(df_metricas_treino_teste_nulos)
                      .union(df_metricas_treino_teste_hiperparametros)
                      .union(df_metricas_treino_teste_multicolinearidade)
                      .union(df_metricas_treino_teste_outlier)
                      .union(df_metricas_treino_teste_stepwise)
                      .union(df_metricas_treino_teste_pvalor)
                      .union(df_metricas_treino_teste_add_variaveis))

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-1756330769975029> in <module> 
 4 df_metricas_treino_teste_hiperparametros = spark . read . csv ( '/FileStore/olist/ENTREGA_3/2_modelos_com_hiperparametros/Comparacao_metricas_treino_teste_hiperparametros.csv' , header = True ) 
 5 df_metricas_gerais = (df_metricas_treino_teste_inicial
 ----> 6 . union ( df_metricas_treino_teste_nulos ) 
 7 . union ( df_metricas_treino_teste_hiperparametros ) 
 8 . union ( df_metricas_treino_teste_multicolinearidade ) 

 NameError : name 'df_metricas_treino_teste_nulos' is not defined

In [104]:
df_metricas_gerais_treino = df_metricas_gerais.filter("base like 'treino%'")
df_metricas_gerais_teste = df_metricas_gerais.filter("base like 'teste%'")

In [105]:
display(df_metricas_gerais_teste)

base,modelo,acuracia,Precision,Recall,underROC,UnderPR
teste,RandomForestClassificationModel,0.8245,0.81908,0.8245,0.7535,0.5794
teste,GBTClassificationModel,0.83171,0.81869,0.83171,0.77077,0.59828
teste,DecisionTreeClassificationModel,0.82963,0.81609,0.82963,0.31268,0.15485
teste,LogisticRegressionModel:,0.82461,0.81563,0.82461,0.76067,0.57653
teste,LinearSVC,0.81345,0.80126,0.81345,0.71321,0.51229
teste_nulos_com_mediana,RandomForestClassificationModel,0.81804,0.82318,0.81804,0.64009,0.45197
teste_nulos_com_mediana,GBTClassificationModel,0.81912,0.82691,0.81912,0.64611,0.45862
teste_nulos_com_mediana,DecisionTreeClassificationModel,0.81768,0.82127,0.81768,0.40131,0.1853
teste_nulos_com_mediana,LogisticRegressionModel:,0.8127,0.81089,0.8127,0.63986,0.43951
teste_nulos_com_mediana,LinearSVC,0.81438,0.8021,0.81438,0.5955,0.41342


As etapas abaixo não foram incluídas na apresentação, sendo executadas em data posterior.

# 7. Resultado Final

##7.1. Recuperar os melhores modelos com hiperparametros

In [109]:
melhorModelo_cross_GBTClassifier = GBTClassificationModel.load('/FileStore/olist/ENTREGA_3/2_modelos_com_hiperparametros/melhorModelo_cross_GBTClassifier1')
print ('Best Param (maxDepth): ', melhorModelo_cross_GBTClassifier._java_obj.getMaxDepth())
print ('Best Param (maxBins): ', melhorModelo_cross_GBTClassifier._java_obj.getMaxBins())
print ('Best Param (minInfoGain): ', melhorModelo_cross_GBTClassifier._java_obj.getMinInfoGain())
print ('Best Param (maxIter): ', melhorModelo_cross_GBTClassifier._java_obj.getMaxIter())

Best Param (maxDepth): 30
Best Param (maxBins): 50
Best Param (minInfoGain): 0.01
Best Param (maxIter): 100

In [110]:
melhorModelo_cross_GBTClassifier = GBTClassifier(labelCol="target",maxDepth=30, maxBins=50,minInfoGain=0.01, maxIter=100)

In [111]:
melhorModelo_RandomForestClassifier = RandomForestClassificationModel.load('/FileStore/olist/ENTREGA_3/2_modelos_com_hiperparametros/melhorModelo_RandomForestClassifier')
print ('Best Param (maxDepth): ', melhorModelo_RandomForestClassifier._java_obj.getMaxDepth())
print ('Best Param (maxBins): ', melhorModelo_RandomForestClassifier._java_obj.getMaxBins())
print ('Best Param (NumTrees): ', melhorModelo_RandomForestClassifier._java_obj.getNumTrees())

Best Param (maxDepth): 20
Best Param (maxBins): 50
Best Param (NumTrees): 50

In [112]:
melhorModelo_RandomForestClassifier = RandomForestClassifier(labelCol="target",maxDepth=20, maxBins=50, numTrees=50)

In [113]:
melhorModelo_cross_LinearSVC = LinearSVCModel.load('/FileStore/olist/ENTREGA_3/2_modelos_com_hiperparametros/melhorModelo_cross_LinearSVC2')
print ('Best Param (regParam): ', melhorModelo_cross_LinearSVC._java_obj.getRegParam())
print ('Best Param (MaxIter): ', melhorModelo_cross_LinearSVC._java_obj.getMaxIter())

Best Param (regParam): 0.01
Best Param (MaxIter): 100

In [114]:
melhorModelo_cross_LinearSVC = LinearSVC(labelCol="target", regParam=0.01, maxIter=100)

In [115]:
melhorModelo_DecisionTreeClassifier =DecisionTreeClassificationModel.load('/FileStore/olist/ENTREGA_3/2_modelos_com_hiperparametros/melhorModelo_DecisionTreeClassifier')
print ('Best Param (maxDepth): ', melhorModelo_DecisionTreeClassifier._java_obj.getMaxDepth())
print ('Best Param (maxBins): ', melhorModelo_DecisionTreeClassifier._java_obj.getMaxBins())
print ('Best Param (minInfoGain): ', melhorModelo_DecisionTreeClassifier._java_obj.getMinInfoGain())
print ('Best Param (impurity): ', melhorModelo_DecisionTreeClassifier._java_obj.getImpurity())

Best Param (maxDepth): 10
Best Param (maxBins): 10
Best Param (minInfoGain): 0.01
Best Param (impurity): entropy

In [116]:
melhorModelo_DecisionTreeClassifier = DecisionTreeClassifier(labelCol="target", maxDepth=10,maxBins=10, minInfoGain=0.01, impurity="entropy")

In [117]:
melhorModelo_LogisticRegression = LogisticRegressionModel.load("/FileStore/olist/ENTREGA_3/2_modelos_com_hiperparametros/melhorModelo_LogisticRegression")
print ('Best Param (regParam): ', melhorModelo_LogisticRegression._java_obj.getRegParam())
print ('Best Param (MaxIter): ', melhorModelo_LogisticRegression._java_obj.getMaxIter())
print ('Best Param (elasticNetParam): ', melhorModelo_LogisticRegression._java_obj.getElasticNetParam())

Best Param (regParam): 0.001
Best Param (MaxIter): 10
Best Param (elasticNetParam): 0.8

In [118]:
melhorModelo_LogisticRegression = LogisticRegression(labelCol="target", regParam=0.001, maxIter=10, elasticNetParam=0.8)

##7.2. Aplicar melhores modelos nas melhorias aplicáveis com melhor ganho

####7.2.1. Aplicar adição de variáveis

In [121]:
treino_add_variaveis = spark.read.parquet('/FileStore/olist/ENTREGA_3/1_modelos_com_melhorias_propostas/treino_add_variaveis.parquet').cache().coalesce(2)
teste_add_variaveis = spark.read.parquet('/FileStore/olist/ENTREGA_3/1_modelos_com_melhorias_propostas/teste_add_variaveis.parquet').cache().coalesce(2)

In [122]:
# Modelo linear  Support Vector Machine (classifier)
modelo_add_variaveis_svm = melhorModelo_cross_LinearSVC.fit(treino_add_variaveis)

# Modelo Gradient-boosted tree classifier
modelo_add_variaveis_gbt = melhorModelo_cross_GBTClassifier.fit(treino_add_variaveis)

In [123]:
# Modelo Random forest classifier
modelo_add_variaveis_randforest = melhorModelo_RandomForestClassifier.fit(treino_add_variaveis)

# Modelo Logistic Regression
modelo_add_variaveis_reglog = melhorModelo_LogisticRegression.fit(treino_add_variaveis)

# Modelo Decision Tree
modelo_add_variaveis_dectree = melhorModelo_DecisionTreeClassifier.fit(treino_add_variaveis)


In [124]:
modelos_add_variaveis = [modelo_add_variaveis_svm,
                         modelo_add_variaveis_gbt,
                         modelo_add_variaveis_randforest,
                         modelo_add_variaveis_reglog,
                         modelo_add_variaveis_dectree]
bases_valor = [treino_add_variaveis,teste_add_variaveis]
nome_modelo = ['treino_add_var_hiper','teste_add_var_hiper']

df_metricas_treino_teste_add_variaveis_com_hiperparametros = df_metricas(modelos_add_variaveis,bases_valor,nome_bases=nome_modelo)
# display(df_metricas_treino_teste_add_variaveis_com_hiperparametros)
df_metricas_treino_teste_add_variaveis_com_hiperparametros.write.format('csv').option('header',True).mode('overwrite').option('sep',',').save('/FileStore/olist/ENTREGA_3/3_modelos_finais/Comparacao_metricas_treino_teste_add_variaveis_com_hiperparametros.csv')

['treino_add_var_hiper', 'LinearSVC', 0.81587, 0.80421, 0.81587, 0.71304, 0.50784]
['treino_add_var_hiper', 'GBTClassificationModel', 0.84559, 0.84136, 0.84559, 0.7687, 0.65098]
['treino_add_var_hiper', 'RandomForestClassificationModel', 0.87683, 0.88041, 0.87683, 0.85894, 0.76248]
['treino_add_var_hiper', 'LogisticRegressionModel:', 0.83045, 0.81803, 0.83045, 0.76066, 0.58173]
['treino_add_var_hiper', 'DecisionTreeClassificationModel', 0.83321, 0.81939, 0.83321, 0.31189, 0.15983]
['teste_add_var_hiper', 'LinearSVC', 0.81207, 0.79886, 0.81207, 0.71588, 0.5106]
['teste_add_var_hiper', 'GBTClassificationModel', 0.82889, 0.81767, 0.82889, 0.74176, 0.58972]
['teste_add_var_hiper', 'RandomForestClassificationModel', 0.83136, 0.81889, 0.83136, 0.76968, 0.59881]
['teste_add_var_hiper', 'LogisticRegressionModel:', 0.82695, 0.81404, 0.82695, 0.75889, 0.57842]
['teste_add_var_hiper', 'DecisionTreeClassificationModel', 0.82684, 0.81215, 0.82684, 0.3131, 0.16275]

####7.2.2. Aplicar tratamento de outlier

In [126]:
treino_outlier = treino_outlier.coalesce(2).cache()#.repartition(2)

In [127]:
# Modelo Decision Tree
modelo_outlier_dectree = melhorModelo_DecisionTreeClassifier.fit(treino_outlier)

In [128]:
# Modelo Gradient-boosted tree classifier
modelo_outlier_gbt = melhorModelo_cross_GBTClassifier.fit(treino_outlier)

In [129]:
# Modelo linear  Support Vector Machine (classifier)
modelo_outlier_svm = melhorModelo_cross_LinearSVC.fit(treino_outlier)



# Modelo Random forest classifier
modelo_outlier_randforest = melhorModelo_RandomForestClassifier.fit(treino_outlier)

# Modelo Logistic Regression
modelo_outlier_reglog = melhorModelo_LogisticRegression.fit(treino_outlier)



#define os modelos que vão ser avaliads
modelos = [modelo_outlier_reglog, 
           modelo_outlier_dectree, 
           modelo_outlier_svm,
           modelo_outlier_randforest,
           modelo_outlier_gbt]
#define as bases que serão avaliadas
bases = [treino_outlier, teste_outlier]
#se não for base treino e teste devemos passar mais um parâmetro com o nome das bases
nome_bases = ['treino_outlier_hiper', 'teste_outlier_hiper']
df_metricas_treino_teste_outlier_hiper =  df_metricas(modelos, bases, nome_bases)
display(df_metricas_treino_teste_outlier_hiper)
df_metricas_treino_teste_outlier_hiper.write.format('csv').option('header',True).mode('overwrite').option('sep',',').save('/FileStore/olist/ENTREGA_3/3_modelos_finais/Comparacao_metricas_treino_teste_outlier_hiper.csv')

base,modelo,acuracia,Precision,Recall,underROC,UnderPR
treino_outlier_hiper,LogisticRegressionModel:,0.83082,0.81756,0.83082,0.76232,0.58258
treino_outlier_hiper,DecisionTreeClassificationModel,0.83151,0.81766,0.83151,0.30902,0.15979
treino_outlier_hiper,LinearSVC,0.8151,0.80289,0.8151,0.72076,0.52169
treino_outlier_hiper,RandomForestClassificationModel,0.86621,0.86668,0.86621,0.8334,0.7211
treino_outlier_hiper,GBTClassificationModel,0.84571,0.84055,0.84571,0.7663,0.64296
teste_outlier_hiper,LogisticRegressionModel:,0.82851,0.81433,0.82851,0.76126,0.58237
teste_outlier_hiper,DecisionTreeClassificationModel,0.82808,0.81349,0.82808,0.3128,0.16086
teste_outlier_hiper,LinearSVC,0.81391,0.80192,0.81391,0.71696,0.51505
teste_outlier_hiper,RandomForestClassificationModel,0.8326,0.82018,0.8326,0.76973,0.59701
teste_outlier_hiper,GBTClassificationModel,0.83134,0.8197,0.83134,0.74396,0.57915


####7.2.3. Aplicar tratamento de multicolinearidade

In [131]:
treino_multicolinearidade = treino_multicolinearidade.coalesce(2).cache()

In [132]:
# Modelo linear  Support Vector Machine (classifier)
modelo_multicolinearidade_svm = melhorModelo_cross_LinearSVC.fit(treino_multicolinearidade)

# Modelo Gradient-boosted tree classifier
modelo_multicolinearidade_gbt = melhorModelo_cross_GBTClassifier.fit(treino_multicolinearidade)

# Modelo Random forest classifier
modelo_multicolinearidade_randforest = melhorModelo_RandomForestClassifier.fit(treino_multicolinearidade)

# Modelo Logistic Regression
modelo_multicolinearidade_reglog = melhorModelo_LogisticRegression.fit(treino_multicolinearidade)

# Modelo Decision Tree
modelo_multicolinearidade_dectree = melhorModelo_DecisionTreeClassifier.fit(treino_multicolinearidade)

#define os modelos que vão ser avaliads
modelos = [modelo_multicolinearidade_reglog, 
           modelo_multicolinearidade_dectree, 
           modelo_multicolinearidade_svm,
           modelo_multicolinearidade_randforest,
           modelo_multicolinearidade_gbt]
#define as bases que serão avaliadas
bases = [treino_multicolinearidade, teste_multicolinearidade]
#se não for base treino e teste devemos passar mais um parâmetro com o nome das bases
nome_bases = ['treino_multicol_hiper', 'teste_multicol_hiper']
df_metricas_treino_teste_multicolinearidade_hiper =  df_metricas(modelos, bases, nome_bases)
df_metricas_treino_teste_multicolinearidade_hiper.write.format('csv').option('header',True).mode('overwrite').option('sep',',').save('/FileStore/olist/ENTREGA_3/3_modelos_finais/Comparacao_metricas_treino_teste_multicolinearidade_hiper.csv')

['treino_multicol_hiper', 'LogisticRegressionModel:', 0.82978, 0.81698, 0.82978, 0.76305, 0.58469]
['treino_multicol_hiper', 'DecisionTreeClassificationModel', 0.83106, 0.81679, 0.83106, 0.30276, 0.15564]
['treino_multicol_hiper', 'LinearSVC', 0.81565, 0.80384, 0.81565, 0.71519, 0.52053]
['treino_multicol_hiper', 'RandomForestClassificationModel', 0.85842, 0.85852, 0.85842, 0.81542, 0.6933]
['treino_multicol_hiper', 'GBTClassificationModel', 0.84525, 0.84099, 0.84525, 0.76508, 0.64138]
['teste_multicol_hiper', 'LogisticRegressionModel:', 0.82873, 0.81591, 0.82873, 0.75749, 0.57691]
['teste_multicol_hiper', 'DecisionTreeClassificationModel', 0.82977, 0.81513, 0.82977, 0.31069, 0.15754]
['teste_multicol_hiper', 'LinearSVC', 0.81259, 0.79958, 0.81259, 0.71079, 0.51137]
['teste_multicol_hiper', 'RandomForestClassificationModel', 0.83354, 0.82305, 0.83354, 0.7644, 0.59328]
['teste_multicol_hiper', 'GBTClassificationModel', 0.83207, 0.82137, 0.83207, 0.74373, 0.58345]

####7.2.4. Aplicar tratamento de multicolinearidade e outlier

In [134]:
trata_outlier = retira_outliers_de_todas_as_variaveis_binarias()
multicolinearidade = seleciona_por_correlacao()
vetorizacao = vetorizar()
base_outlier_multicolinearidade = vetorizacao.transform(multicolinearidade.transform(trata_outlier.transform(df_tabelao_fillna_zero)))
#base_teste_multicolinearidade_outlier = vetorizacao.transform(trata_outlier.transform(multicolinearidade.transform(df_tabelao_fillna_zero)))
treino_outlier_multicolinearidade, teste_outlier_multicolinearidade = base_outlier_multicolinearidade.randomSplit([0.7, 0.3], seed=42)

treino_outlier_multicolinearidade = treino_outlier_multicolinearidade.cache().coalesce(2)#.repartition(2)
teste_outlier_multicolinearidade = teste_outlier_multicolinearidade.cache().coalesce(2)#.repartition(2)
treino_outlier_multicolinearidade.write.mode("overwrite").parquet('/FileStore/olist/ENTREGA_3/1_modelos_com_melhorias_propostas/treino_outlier_multicolinearidade.parquet')
teste_outlier_multicolinearidade.write.mode("overwrite").parquet('/FileStore/olist/ENTREGA_3/1_modelos_com_melhorias_propostas/teste_outlier_multicolinearidade.parquet')

***** Análise de outliers *****
	A coluna soma_orders_2_trimestre_2016 foi removida!
	A coluna soma_orders_3_trimestre_2016 possui pelo menos 90% base com a categoria 0 !
	A coluna soma_orders_1_trimestre_2016 foi removida!
	A coluna soma_orders_4_trimestre_2018 foi removida!
	A coluna soma_orders_1_semestre_2016 foi removida!
***** Fim Análise de outliers *****
***** Análise de correlação *****
Colunas deletadas:
	soma_orders_approved_year_2016
	soma_orders_4_trimestre_2016
	soma_orders_3_trimestre_2018
	media_dias_entre_delivered_e_review
	media_review_comment_message_length
	media_review_comment_title_length
***** Fim Análise de correlação *****

In [135]:
# Modelo linear  Support Vector Machine (classifier)
modelo_mult_outlier_svm = melhorModelo_cross_LinearSVC.fit(treino_outlier_multicolinearidade)

# Modelo Gradient-boosted tree classifier
modelo_mult_outlier_gbt = melhorModelo_cross_GBTClassifier.fit(treino_outlier_multicolinearidade)

# Modelo Random forest classifier
modelo_mult_outlier_randforest = melhorModelo_RandomForestClassifier.fit(treino_outlier_multicolinearidade)

# Modelo Logistic Regression
modelo_mult_outlier_reglog = melhorModelo_LogisticRegression.fit(treino_outlier_multicolinearidade)

# Modelo Decision Tree
modelo_mult_outlier_dectree = melhorModelo_DecisionTreeClassifier.fit(treino_outlier_multicolinearidade)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-3246182700756838> in <module> 
 21 modelo_mult_outlier_gbt]
 22 #define as bases que serão avaliadas 
 ---> 23 bases = [ treino_mult_outlier , teste_mult_outlier ] 
 24 #se não for base treino e teste devemos passar mais um parâmetro com o nome das bases 
 25 nome_bases = [ 'treino_mult_outlier_hiper' , 'teste_mult_outlier_hiper' ] 

 NameError : name 'treino_mult_outlier' is not defined

In [136]:
#define os modelos que vão ser avaliads
modelos = [modelo_mult_outlier_reglog, 
           modelo_mult_outlier_dectree, 
           modelo_mult_outlier_svm,
           modelo_mult_outlier_randforest,
           modelo_mult_outlier_gbt]
#define as bases que serão avaliadas
bases = [treino_outlier_multicolinearidade, teste_outlier_multicolinearidade]
#se não for base treino e teste devemos passar mais um parâmetro com o nome das bases
nome_bases = ['treino_mult_outlier_hiper', 'teste_mult_outlier_hiper']
df_metricas_treino_teste_mult_outlier_hiper =  df_metricas(modelos, bases, nome_bases)
df_metricas_treino_teste_mult_outlier_hiper.write.format('csv').option('header',True).mode('overwrite').option('sep',',').save('/FileStore/olist/ENTREGA_3/3_modelos_finais/Comparacao_metricas_treino_teste_mult_outlier_hiper.csv')

['treino_mult_outlier_hiper', 'LogisticRegressionModel:', 0.82993, 0.81668, 0.82993, 0.76213, 0.58396]
['treino_mult_outlier_hiper', 'DecisionTreeClassificationModel', 0.83103, 0.81677, 0.83103, 0.30451, 0.15609]
['treino_mult_outlier_hiper', 'LinearSVC', 0.81548, 0.80372, 0.81548, 0.72201, 0.52249]
['treino_mult_outlier_hiper', 'RandomForestClassificationModel', 0.85958, 0.85993, 0.85958, 0.81834, 0.70001]
['treino_mult_outlier_hiper', 'GBTClassificationModel', 0.84504, 0.84088, 0.84504, 0.76445, 0.6413]
['teste_mult_outlier_hiper', 'LogisticRegressionModel:', 0.82887, 0.81524, 0.82887, 0.75874, 0.57779]
['teste_mult_outlier_hiper', 'DecisionTreeClassificationModel', 0.8307, 0.81619, 0.8307, 0.30884, 0.1566]
['teste_mult_outlier_hiper', 'LinearSVC', 0.81298, 0.79983, 0.81298, 0.71985, 0.5144]
['teste_mult_outlier_hiper', 'RandomForestClassificationModel', 0.83403, 0.82311, 0.83403, 0.7672, 0.59597]
['teste_mult_outlier_hiper', 'GBTClassificationModel', 0.83199, 0.82119, 0.83199, 0.74545, 0.5816]

##7.3. Avaliar melhores modelos nas melhorias aplicáveis com melhor ganho

In [138]:
df_metricas_treino_teste_hiperparametros = spark.read.csv('/FileStore/olist/ENTREGA_3/2_modelos_com_hiperparametros/Comparacao_metricas_treino_teste_hiperparametros.csv',header=True)

In [139]:
df_metricas_treino_teste_multicolinearidade = spark.read.csv('/FileStore/olist/ENTREGA_3/1_modelos_com_melhorias_propostas/1_multicolinearidade/Comparacao_metricas_treino_teste_multicolinearidade.csv',header=True)

In [140]:
df_metricas_treino_teste_outlier = spark.read.csv('/FileStore/olist/ENTREGA_3/1_modelos_com_melhorias_propostas/2_outliers/Comparacao_metricas_treino_teste_outlier.csv',header=True)

In [141]:
df_metricas_treino_teste_add_variaveis = spark.read.csv('/FileStore/olist/ENTREGA_3/1_modelos_com_melhorias_propostas/5_novas_variaveis/Comparacao_metricas_treino_teste_add_variaveis.csv', header=True)

In [142]:
df_metricas_treino_teste_add_variaveis_com_hiperparametros = spark.read.csv('/FileStore/olist/ENTREGA_3/3_modelos_finais/Comparacao_metricas_treino_teste_add_variaveis_com_hiperparametros.csv', header=True)

In [143]:
df_metricas_treino_teste_outlier_hiper = spark.read.csv('/FileStore/olist/ENTREGA_3/3_modelos_finais/Comparacao_metricas_treino_teste_outlier_hiper.csv', header=True)

In [144]:
df_metricas_treino_teste_mult_outlier_hiper = spark.read.csv('/FileStore/olist/ENTREGA_3/3_modelos_finais/Comparacao_metricas_treino_teste_mult_outlier_hiper.csv', header=True)

In [145]:
df_metricas_treino_teste_multicolinearidade_hiper = spark.read.csv('/FileStore/olist/ENTREGA_3/3_modelos_finais/Comparacao_metricas_treino_teste_multicolinearidade_hiper.csv', header=True)

In [146]:
df_metricas_finais = (df_metricas_treino_teste_inicial
                      .union(df_metricas_treino_teste_multicolinearidade)
                      .union(df_metricas_treino_teste_outlier)
                      .union(df_metricas_treino_teste_add_variaveis)
                      .union(df_metricas_treino_teste_hiperparametros)
                      .union(df_metricas_treino_teste_add_variaveis_com_hiperparametros)
                      .union(df_metricas_treino_teste_outlier_hiper)
                      .union(df_metricas_treino_teste_multicolinearidade_hiper)
                      .union(df_metricas_treino_teste_mult_outlier_hiper))

df_metricas_finais_teste = df_metricas_finais.filter("base like 'teste%'")

In [147]:
display(df_metricas_finais_teste)

base,modelo,acuracia,Precision,Recall,underROC,UnderPR
teste,RandomForestClassificationModel,0.8245,0.81908,0.8245,0.7535,0.5794
teste,GBTClassificationModel,0.83171,0.81869,0.83171,0.77077,0.59828
teste,DecisionTreeClassificationModel,0.82963,0.81609,0.82963,0.31268,0.15485
teste,LogisticRegressionModel:,0.82461,0.81563,0.82461,0.76067,0.57653
teste,LinearSVC,0.81345,0.80126,0.81345,0.71321,0.51229
teste_multicolinearidade,RandomForestClassificationModel,0.81826,0.82657,0.81826,0.74798,0.57266
teste_multicolinearidade,GBTClassificationModel,0.83332,0.821,0.83332,0.76841,0.59677
teste_multicolinearidade,DecisionTreeClassificationModel,0.83027,0.81747,0.83027,0.34077,0.16007
teste_multicolinearidade,LogisticRegressionModel:,0.82148,0.81735,0.82148,0.75783,0.57562
teste_multicolinearidade,LinearSVC,0.81259,0.79958,0.81259,0.71933,0.51603


In [148]:
df_metricas_tunadas = (df_metricas_treino_teste_inicial
                      .union(df_metricas_treino_teste_add_variaveis_com_hiperparametros)
                      .union(df_metricas_treino_teste_outlier_hiper)
                      .union(df_metricas_treino_teste_multicolinearidade_hiper)
                      .union(df_metricas_treino_teste_mult_outlier_hiper))

df_metricas_tunadas_teste = df_metricas_tunadas.filter("base like 'teste%'")

In [149]:
display(df_metricas_tunadas_teste)

base,modelo,acuracia,Precision,Recall,underROC,UnderPR
teste,RandomForestClassificationModel,0.8245,0.81908,0.8245,0.7535,0.5794
teste,GBTClassificationModel,0.83171,0.81869,0.83171,0.77077,0.59828
teste,DecisionTreeClassificationModel,0.82963,0.81609,0.82963,0.31268,0.15485
teste,LogisticRegressionModel:,0.82461,0.81563,0.82461,0.76067,0.57653
teste,LinearSVC,0.81345,0.80126,0.81345,0.71321,0.51229
teste_add_var_hiper,LogisticRegressionModel:,0.82695,0.81404,0.82695,0.75889,0.57842
teste_add_var_hiper,DecisionTreeClassificationModel,0.82684,0.81215,0.82684,0.3131,0.16275
teste_add_var_hiper,RandomForestClassificationModel,0.83136,0.81889,0.83136,0.76968,0.59881
teste_add_var_hiper,GBTClassificationModel,0.82889,0.81767,0.82889,0.74176,0.58972
teste_add_var_hiper,LinearSVC,0.81207,0.79886,0.81207,0.71588,0.5106


Esta comparação final nos mostra que os melhores modelos foram RandomForestClassifier e GBTClassifier.
Isto modifica a conclusão mostrada anteriormente na apresentação